<div align="center">
    <img src="https://www.sharif.ir/documents/20124/0/logo-fa-IR.png/4d9b72bc-494b-ed5a-d3bb-e7dfd319aec8?t=1609608338755" alt="Logo" width="200">
    <p><b>HW1 @ Deep Learning Course, Dr. Soleymani</b></p>
    <p><b>ِDesinged by Payam Taebi</b></p>
</div>

---




*Full Name:* Nikan Vasei

*Student Number:* 400105303



# Overview(100 Points)

This notebook is primarily for exploration rather than heavy implementation. We aim to examine the performance of various optimization methods on different functions. The submission for this exercise is in-person, so please experiment with the parameters in this section to better understand the challenges of each part.

We will define several 1D and 2D functions (e.g., Rastrigin, Schwefel, Rosenbrock, Ackley, Beale, Eggholder, Ill-Conditioned Convex functions) using PyTorch. These functions serve as challenging test cases for our optimizers.

Implement different optimization algorithms:
We cover first-order methods (SGD, SGD with Momentum, Nesterov Accelerated Gradient, Adagrad, RMSprop, Adam, Nadam) and second-order methods (Newton's Method, L-BFGS). Each method's update rule is implemented and explained.

These are some of the components we used.


# First Section

In the first section, we have a set of functions for demonstration. Do not modify these; they are implemented to display the functions in this section.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import plotly.graph_objects as go
def plot_1d_function_torch(func, domain, global_x, title="1D Function Plot", initial_x=None, path=None):
    """
    Plot a 1D function (defined in PyTorch) and mark the global minimum.
    Optionally, mark an initial point and the optimization path with directional arrows.

    Parameters:
      - func: a function that accepts a torch tensor and returns a torch tensor.
      - domain: tuple (xmin, xmax)
      - global_x: x-coordinate of the global minimum.
      - title: title for the plot.
      - initial_x: (optional) x-coordinate of the initial point.
      - path: (optional) a list or numpy array of x-values representing the optimization path.
    """
    x_np = np.linspace(domain[0], domain[1], 1000)
    x_torch = torch.tensor(x_np, dtype=torch.float32)
    with torch.no_grad():
        y_torch = func(x_torch)
    y_np = y_torch.numpy()

    plt.figure(figsize=(8, 4))
    plt.plot(x_np, y_np, lw=2, label=title)

    # Mark initial point if provided.
    if initial_x is not None:
        init_val = func(torch.tensor(initial_x, dtype=torch.float32)).item()
        plt.scatter(initial_x, init_val, color='blue', s=80, zorder=5, label='Initial Point')

    # Plot the optimization path if provided.
    if path is not None:
        path = np.array(path)
        # Draw a line connecting the points.
        y_path = [func(torch.tensor(p, dtype=torch.float32)).item() for p in path]
        plt.plot(path, y_path, color='orange', linewidth=2, marker='o', markersize=4, label='Optimization Path')
        # Draw arrows for each segment.
        for i in range(len(path)-1):
            start = path[i]
            end = path[i+1]
            y_start = func(torch.tensor(start, dtype=torch.float32)).item()
            y_end = func(torch.tensor(end, dtype=torch.float32)).item()
            plt.annotate("",
                         xy=(end, y_end),
                         xytext=(start, y_start),
                         arrowprops=dict(arrowstyle="->", color='orange', lw=2))

    # Mark the global minimum.
    global_val = func(torch.tensor(global_x, dtype=torch.float32)).item()
    plt.scatter(global_x, global_val, color='red', s=80, zorder=5, label='Global Minimum')

    plt.title(title)
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()
    plt.grid(True)
    plt.show()
def plot_2d_contour_torch(func, x_domain, y_domain, global_point, title="2D Function Contour",
                            levels=50, cmap='viridis', initial_point=None, path=None):
    """
    Plot a 2D function (defined in PyTorch) using a contour plot, marking the global minimum.
    Optionally, mark an initial point and draw the optimization path with arrows indicating direction.

    Parameters:
      - func: a function that accepts a torch tensor of shape (2, H, W) and returns a tensor of shape (H, W).
      - x_domain: tuple (xmin, xmax)
      - y_domain: tuple (ymin, ymax)
      - global_point: tuple (x*, y*) of the global minimum.
      - title: title for the plot.
      - levels: number of contour levels.
      - cmap: colormap.
      - initial_point: (optional) tuple (x0, y0) for the initial point.
      - path: (optional) numpy array of shape (n, 2) representing the optimization path.
    """
    x_np = np.linspace(x_domain[0], x_domain[1], 400)
    y_np = np.linspace(y_domain[0], y_domain[1], 400)
    X_np, Y_np = np.meshgrid(x_np, y_np)

    # Create input grid tensor with shape (2, H, W)
    grid = torch.tensor(np.stack([X_np, Y_np], axis=0), dtype=torch.float32)
    with torch.no_grad():
        Z_torch = func(grid)
    Z_np = Z_torch.numpy()
    vmin, vmax = Z_np.min(), Z_np.max()

    plt.figure(figsize=(10, 8))
    cp = plt.contourf(X_np, Y_np, Z_np, levels=levels, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.colorbar(cp)
    plt.contour(X_np, Y_np, Z_np, levels=15, colors='black', alpha=0.5)

    # Mark initial point if provided.
    if initial_point is not None:
        plt.scatter(initial_point[0], initial_point[1], color='blue', s=100, marker='o', label='Initial Point')

    # Plot the optimization path if provided.
    if path is not None:
        path = np.array(path)
        plt.plot(path[:, 0], path[:, 1], marker='o', color='orange', markersize=4, linewidth=2, label='Optimization Path')
        for i in range(len(path) - 1):
            start = path[i]
            end = path[i+1]
            plt.annotate("",
                         xy=(end[0], end[1]),
                         xytext=(start[0], start[1]),
                         arrowprops=dict(arrowstyle="->", color='orange', lw=2))

    # Mark the global minimum.
    plt.scatter(global_point[0], global_point[1], color='red', s=100, marker='*', label='Global Minimum')

    plt.title(title + " (Contour Plot)")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.show()
def plot_interactive_3d_torch(func, x_domain, y_domain, global_point, title="Interactive 3D Plot",
                              colorscale='viridis', initial_point=None, path=None):
    """
    Create an interactive 3D surface plot using Plotly for a 2D function defined in PyTorch.
    Assumes the input is a tensor with two channels: first channel for x and second for y.
    Optionally, marks an initial point and draws cones (arrows) to indicate direction along the optimization path.

    Parameters:
      - func: a function that accepts a torch tensor of shape (2, H, W) and returns a tensor of shape (H, W).
      - x_domain: tuple (xmin, xmax)
      - y_domain: tuple (ymin, ymax)
      - global_point: tuple (x*, y*) for the global minimum.
      - title: title for the plot.
      - colorscale: Plotly colorscale.
      - initial_point: (optional) tuple (x0, y0) for the initial point.
      - path: (optional) numpy array of shape (n, 2) representing the optimization path.
    """
    # Create grid for the surface.
    x_np = np.linspace(x_domain[0], x_domain[1], 200)
    y_np = np.linspace(y_domain[0], y_domain[1], 200)
    X_np, Y_np = np.meshgrid(x_np, y_np)

    # Create grid tensor with shape (2, H, W)
    grid = torch.tensor(np.stack([X_np, Y_np], axis=0), dtype=torch.float32)
    with torch.no_grad():
        Z_torch = func(grid)
    Z_np = Z_torch.numpy()

    surface = go.Surface(x=X_np, y=Y_np, z=Z_np, colorscale=colorscale, opacity=0.9, name=title)
    data = [surface]

    # Optionally add initial point.
    if initial_point is not None:
        init_val = func(torch.tensor([initial_point[0], initial_point[1]], dtype=torch.float32)).item()
        scatter_initial = go.Scatter3d(
            x=[initial_point[0]],
            y=[initial_point[1]],
            z=[init_val],
            mode='markers',
            marker=dict(size=6, color='blue'),
            name='Initial Point'
        )
        data.append(scatter_initial)

    # Optionally add optimization path.
    if path is not None:
        path = np.array(path)
        z_path = []
        for pt in path:
            z_val = func(torch.tensor([pt[0], pt[1]], dtype=torch.float32)).item()
            z_path.append(z_val)
        scatter_path = go.Scatter3d(
            x=path[:, 0],
            y=path[:, 1],
            z=z_path,
            mode='lines+markers',
            line=dict(color='orange', width=4),
            marker=dict(size=4, color='orange'),
            name='Optimization Path'
        )
        data.append(scatter_path)

        # Add cone traces for directional arrows.
        cone_x = []
        cone_y = []
        cone_z = []
        cone_u = []
        cone_v = []
        cone_w = []
        for i in range(len(path) - 1):
            start = path[i]
            end = path[i+1]
            cone_x.append(start[0])
            cone_y.append(start[1])
            # Evaluate z at start.
            z_start = func(torch.tensor([start[0], start[1]], dtype=torch.float32)).item()
            cone_z.append(z_start)
            cone_u.append(end[0] - start[0])
            cone_v.append(end[1] - start[1])
            z_end = func(torch.tensor([end[0], end[1]], dtype=torch.float32)).item()
            cone_w.append(z_end - z_start)
        if len(cone_x) > 0:
            cone_trace = go.Cone(
                x=cone_x,
                y=cone_y,
                z=cone_z,
                u=cone_u,
                v=cone_v,
                w=cone_w,
                sizemode="absolute",
                sizeref=0.5,
                anchor="tail",
                showscale=False,
                colorscale=[[0, 'orange'], [1, 'orange']],
                name="Step Arrows"
            )
            data.append(cone_trace)

    # Add global minimum.
    global_tensor = torch.tensor([global_point[0], global_point[1]], dtype=torch.float32)
    with torch.no_grad():
        global_z = func(global_tensor).item()
    scatter_global = go.Scatter3d(
        x=[global_point[0]],
        y=[global_point[1]],
        z=[global_z],
        mode='markers',
        marker=dict(size=8, color='red', symbol='diamond'),
        name='Global Minimum'
    )
    data.append(scatter_global)

    fig = go.Figure(data=data)
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='x',
            yaxis_title='y',
            zaxis_title='f(x,y)'
        ),
        autosize=True
    )
    fig.show()


def plot_loss_vs_epoch(loss_func, path):

    """
    Given a loss function and an optimization path, compute the loss at each epoch and plot it.

    Parameters:
      - loss_func: A function that accepts a torch tensor representing parameters and returns a scalar loss.
      - path: A list or NumPy array of parameter values (e.g. from the optimizer).
              Each element should be convertible to a torch tensor.
    """
    losses = []
    # Loop over each parameter value in the path.
    for p in path:
        # Convert parameter p to a torch tensor (assume float32).
        # p can be a 1D array or a multi-dimensional array.
        p_tensor = torch.tensor(p, dtype=torch.float32)
        with torch.no_grad():
            loss_val = loss_func(p_tensor).item()
        losses.append(loss_val)

    epochs = np.arange(len(losses))

    plt.figure(figsize=(8, 4))
    plt.plot(epochs, losses, marker='o', color='blue', linewidth=2)
    plt.title("Loss vs. Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.show()



# Next Section (18 Points)

In the following section, we examine all the functions, each of which presents its own unique challenge. These functions are both one-dimensional and two-dimensional.


# 1D Rastrigin Function

**Mathematical Definition:**

$$
f(x) = x^2 - 10 \cos\left(2\pi x\right) + 10
$$

**Domain:**  
\\( x \in [-5,\, 5] \\)

**Global Minimum:**  
\\( x = 0 \\) with \\( f(0)=0 \\)

**Explanation:**  
This function is **non-convex** and exhibits many local minima due to the cosine modulation. Its oscillatory behavior over \\( [-5,5] \\) makes it a popular test for global optimization methods.


In [ ]:
# 1D Rastrigin function in PyTorch.
def rastrigin_1d_torch(x: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 1D Rastrigin function using PyTorch operations
    return x**2 - 10 * torch.cos(2 * torch.pi * x) + 10

# Plot the 1D Rastrigin function.
plot_1d_function_torch(rastrigin_1d_torch, [-5, 5], global_x=0, title="1D Rastrigin Function")

# 1D Schwefel Function

**Mathematical Definition:**

$$
f(x) = 418.9829 - x \sin\Bigl(\sqrt{|x|}\Bigr)
$$

**Domain:**  
\\( x \in [0,\, 500] \\)

**Global Minimum:**  
Approximately at \\( x \approx 420.9687 \\)

**Explanation:**  
This function is highly **non-convex** and rugged with many deceptive local minima. Its large domain accentuates the difficulty in escaping local traps, making it a challenging benchmark for optimization.


In [ ]:
# 1D Schwefel function in PyTorch.
def schwefel_1d_torch(x: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 1D Schwefel function using PyTorch operations
    return 418.9829 - x * torch.sin(torch.sqrt(torch.abs(x)))

# Plot the 1D Schwefel function.
plot_1d_function_torch(schwefel_1d_torch, [0, 500], global_x=420.9687, title="1D Schwefel Function")


# 1D Ill-Conditioned Convex Function

**Mathematical Definition:**

$$
f(x) = \ln\Bigl(1 + e^{10x}\Bigr) + x^2
$$

**Domain:**  
\\( x \in [-5,\, 5] \\)

**Global Minimum:**  
Approximately at \\( x \approx -0.3 \\)

**Explanation:**  
This function is **convex** (and therefore has a unique global minimum), but it is _ill-conditioned_ because the steep exponential part for \\( x > 0 \\) creates a rapidly changing gradient. It tests how well an optimizer can adjust its step size.


In [ ]:
# 1D Ill-Conditioned Convex function in PyTorch.
def ill_conditioned_convex_1d_torch(x: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 1D ill-conditioned convex function using PyTorch operations
    return torch.log(1 + torch.exp(10 * x)) + x**2

# Plot the 1D Ill-Conditioned Convex function.
plot_1d_function_torch(ill_conditioned_convex_1d_torch, [-5, 5], global_x=-0.3, title="1D Ill-Conditioned Convex Function")


# Rosenbrock Function (2D)

**Mathematical Definition:**

$$
f(x, y) = (1 - x)^2 + 100 \,(y - x^2)^2
$$

**Domain:**  
\\( x \in [-2,\, 2] \\) and \\( y \in [-1,\, 3] \\)

**Global Minimum:**  
At \\( (x,y) = (1,1) \\) with \\( f(1,1)=0 \\)

**Explanation:**  
The Rosenbrock function is **non-convex** despite its smoothness. Its narrow, curved valley makes it challenging for gradient-based optimizers to converge to the global minimum. This function is a classical benchmark for testing optimization algorithms.

In the next cells, we plot it both as a static contour and as an interactive 3D plot. We'll also mark an initial point (for example, \\( (-1,1.5) \\)) and the optimization path.


In [ ]:
# Define the Rosenbrock function for 2D inputs in PyTorch.
# The input is a tensor of shape (2, H, W) where the first channel is x and the second is y.
def rosenbrock_2d_torch(X: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 2D Rosenbrock function using PyTorch operations
    return (1 - X[0]) ** 2 + 100 * (X[1] - X[0]**2) ** 2

# Plot using static contour.
plot_2d_contour_torch(rosenbrock_2d_torch, [-2, 2], [-1, 3], global_point=(1, 1),
                      title="Rosenbrock Function (2D)")

# Plot using interactive 3D.
plot_interactive_3d_torch(rosenbrock_2d_torch, [-2, 2], [-1, 3], global_point=(1, 1),
                          title="Interactive 3D Rosenbrock Function")


# 2D Rastrigin Function

**Mathematical Definition:**

$$
f(x, y) = x^2 + y^2 - 10 \Bigl[\cos\left(2\pi x\right) + \cos\left(2\pi y\right)\Bigr] + 20
$$

**Domain:**  
\\( x, y \in [-5,\, 5] \\)

**Global Minimum:**  
At \\( (0,0) \\) with \\( f(0,0)=0 \\)

**Explanation:**  
The 2D Rastrigin function is highly **non-convex** and multimodal, featuring many local minima. Its oscillatory behavior in both dimensions makes it a demanding test for optimization techniques.

Below, we display both the static contour and interactive 3D plots with an example initial point (e.g., \\( (3,3) \\)) and a dummy optimization path.


In [ ]:
# Define the 2D Rastrigin function in PyTorch.
def rastrigin_2d_torch(X: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 2D Rastrigin function using PyTorch operations
    return X[0]**2 + X[1]**2 - 10 * (torch.cos(2 * torch.pi * X[0]) + torch.cos(2 * torch.pi * X[1])) + 20

# Plot as a static contour.
plot_2d_contour_torch(rastrigin_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                      title="2D Rastrigin Function", cmap='plasma')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(rastrigin_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                          title="Interactive 3D Rastrigin Function", colorscale='plasma')


# Ackley Function (2D)

**Mathematical Definition:**

$$
f(x, y) = -20 \exp\!\left(-0.2 \sqrt{\frac{x^2+y^2}{2}}\right) - \exp\!\left(\frac{\cos\left(2\pi x\right)+\cos\left(2\pi y\right)}{2}\right) + 20 + e
$$

**Domain:**  
\\( x, y \in [-5,\, 5] \\)

**Global Minimum:**  
At \\( (0,0) \\) with \\( f(0,0)=0 \\)

**Explanation:**  
The Ackley function features a nearly flat outer region with a deep, narrow central pit, creating a challenging landscape for optimization algorithms. This function is popular for testing global optimization techniques.

We now show both the contour and interactive 3D plots with an initial point (e.g., \\( (3,3) \\)) and an optimization path.


In [ ]:
# Define the 2D Ackley function in PyTorch.
def ackley_2d_torch(X: torch.Tensor, a=20, b=0.2, c=2*np.pi) -> torch.Tensor:
    # TODO: Implement the 2D Ackley function using PyTorch operations
    return -20 * torch.exp(-0.2 * torch.sqrt((X[0]**2 + X[1]**2) / 2)) - torch.exp((torch.cos(2 * torch.pi * X[0]) + torch.cos(2 * torch.pi * X[1])) / 2) + 20 + torch.e

# Plot as a static contour.
plot_2d_contour_torch(ackley_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                      title="Ackley Function (2D)", cmap='inferno')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(ackley_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                          title="Interactive 3D Ackley Function", colorscale='inferno')


# Beale Function (2D)

**Mathematical Definition:**

$$
f(x, y) = \left(1.5 - x + xy\right)^2 + \left(2.25 - x + xy^2\right)^2 + \left(2.625 - x + xy^3\right)^2
$$

**Domain:**  
\\( x, y \in [-4.5,\, 4.5] \\)

**Global Minimum:**  
At \\( (3, 0.5) \\) with \\( f(3,0.5)=0 \\)

**Explanation:**  
The Beale function is **non-convex** with several local minima, making it a classical benchmark for global optimization. Its complex structure challenges optimizers in 2D space.

Below, we provide both a static contour and an interactive 3D plot with an initial point (e.g., \\( (0,0) \\)) and an optimization path.


In [ ]:
# Define the 2D Beale function in PyTorch.
def beale_2d_torch(X: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 2D Beale function using PyTorch operations
    return (1.5 - X[0] + X[0] * X[1]) ** 2 + (2.25 - X[0] + X[0] * X[1]) ** 2 + (2.625 - X[0] + X[0] * X[1] ** 3) ** 2

# Plot as a static contour.
plot_2d_contour_torch(beale_2d_torch, [-4.5, 4.5], [-4.5, 4.5], global_point=(3, 0.5),
                      title="Beale Function (2D)", cmap='Spectral')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(beale_2d_torch, [-4.5, 4.5], [-4.5, 4.5], global_point=(3, 0.5),
                          title="Interactive 3D Beale Function", colorscale='Spectral')


# Eggholder Function (2D)

**Mathematical Definition:**

$$
f(x, y) = - (y+47) \sin\!\left(\sqrt{\left|\frac{x}{2} + (y+47)\right|}\right) - x \sin\!\left(\sqrt{\left|x - (y+47)\right|}\right)
$$

**Domain:**  
\\( x, y \in [-512,\, 512] \\)

**Global Minimum:**  
Approximately at \\( (512,\, 404.2319) \\)

**Explanation:**  
The Eggholder function is extremely **non-convex** with a rugged, highly oscillatory landscape that contains many local minima. It is one of the most challenging test functions in optimization.


In [ ]:
# Define the 2D Eggholder function in PyTorch.
def eggholder_2d_torch(X: torch.Tensor) -> torch.Tensor:
    # TODO: Implement the 2D Eggholder function using PyTorch operations
    return -(X[1] + 47) * torch.sin(torch.sqrt(torch.abs(X[0] / 2 + (X[1] + 47)))) - X[0] * torch.sin(torch.sqrt(torch.abs(X[0] - (X[1] + 47))))

# Plot as a static contour.
plot_2d_contour_torch(eggholder_2d_torch, [-512, 512], [-512, 512], global_point=(512, 404.2319),
                      title="Eggholder Function (2D)", cmap='inferno')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(eggholder_2d_torch, [-512, 512], [-512, 512], global_point=(512, 404.2319),
                          title="Interactive 3D Eggholder Function", colorscale='inferno')


# Ill-Conditioned Convex Function (2D)

**Mathematical Definition:**

$$
f(x, y) = 100 \Bigl(x\cos\theta - y\sin\theta\Bigr)^2 + \Bigl(x\sin\theta + y\cos\theta\Bigr)^2 \quad \text{with } \theta = \frac{\pi}{6}
$$

**Domain:**  
\\( x, y \in [-5,\, 5] \\)

**Global Minimum:**  
At \\( (0,0) \\) with \\( f(0,0)=0 \\)

**Explanation:**  
This function is **convex** (thus it has a unique global minimum) but is _ill-conditioned_ due to the rotation and different scaling along the rotated axes. Its behavior tests an optimizer’s ability to deal with steep versus flat directions.


In [ ]:
# Define the 2D Ill-Conditioned Convex function in PyTorch.
def ill_conditioned_convex_2d_torch(X: torch.Tensor, theta=torch.tensor(np.pi/6)) -> torch.Tensor:
    # TODO: Implement the 2D ill-conditioned convex function using PyTorch operations
    return 100 * (X[0] * torch.cos(theta) - X[1] * torch.sin(theta)) ** 2 + (X[0] * torch.sin(theta) - X[1] * torch.cos(theta)) ** 2

# Plot as a static contour.
plot_2d_contour_torch(ill_conditioned_convex_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                      title="Ill-Conditioned Convex Function (2D)", cmap='viridis')

# Plot as an interactive 3D surface.
plot_interactive_3d_torch(ill_conditioned_convex_2d_torch, [-5, 5], [-5, 5], global_point=(0, 0),
                          title="Interactive 3D Ill-Conditioned Convex Function (2D)", colorscale='viridis')


# Question (12 Points)

For each of the functions presented above, what unique challenges do you think they offer? Consider aspects such as steep gradients, saddle points, local minima, flat regions, and other characteristics that could affect the optimization process.

Please provide a brief explanation for each function, describing the potential difficulties in learning or optimizing it.


## Answer

1. ### 1D Rastrigin Function
   - Non-convex surface
   - Multiple local minima due to the oscillatory cosine term (Traps gradient-based methods away from the global optimum)
   - Steep valleys and flat regions (Makes step-size selection tricky)
   - Sensitivity to initialization (Where you start heavily influences which minima you find)

1. ### 1D Schwefel Function
   - Non-convex surface
   - Numerous local minima (“trap” points along the domain)
   - Highly rugged landscape (Rapid fluctuations in the function’s slope can disrupt smooth convergence)
   - Large domain (A wide search interval increases the chance of getting stuck far from the global minimum)
   - Steep slopes near certain regions (Can cause overshooting or unstable steps)

1. ### 1D Ill-Conditioned Convex Function
   - Ill-conditioning (Leads to slow or unstable convergence)
   - Rapidly changing gradient (Can produce large updates in certain regions)
   - Potential numerical stability issues

1. ### 2D Rosenbrock Function
   - Non-convex surface
   - Long, narrow valley (Requires precise steps to avoid bouncing or stalling)
   - Slow convergence near the valley
   - Sensitivity to hyperparameters

1. ### 2D Rastrigin Function
   - Non-convex surface
   - Multidimensional oscillations (The function repeats peaks and valleys along both x and y)
   - Abundance of local minima
   - Steep drop-offs and gentle plateaus

1. ### 2D Ackley Function
   - Non-convex with oscillatory components
   - Wide flat region (Far from the origin, gradients are tiny and slow to guide the optimizer)
   - Sharp basin around the global minimum (Close to zero, the function drops quickly and can overshoot)
   - Sensitivity to initialization

1. ### 2D Beale Function
   - Non-convex surface
   - Multiple local minima
   - Complex interaction of terms
   - Steep vs. flat regions

1. ### 2D Eggholder Function
   - Non-convex surface
   - Highly oscillatory and multi-modal (Numerous peaks and dips produce many local optima)
   - Large domain
   - Complex gradient behavior

1. ### 2D Ill-Conditioned Convex Function
   - Convex yet elongated “valley” (Only one global minimum, but reaching it requires careful maneuvering)
   - Severe ill-conditioning
   - Sensitivity to step-size/learning-rate tuning



Overall we can conclude that optimizers encounter these sets of challenges in different scenarios (in general):

- **Non-convexity and Multiple Local Minima** (e.g., Rastrigin, Schwefel, Ackley, Eggholder) show how easily an algorithm can be trapped away from the global optimum.
- **Ill-Conditioning** (e.g., the Ill-Conditioned Convex functions) tests the optimizer’s ability to handle skewed or elongated “valleys” where gradients may vary dramatically in different directions.
- **Rugged Landscapes and Oscillatory Behavior** (e.g., Eggholder, Rastrigin, Schwefel) underscore the importance of careful step-size/learning-rate strategies to avoid hopping between—or getting stuck in—local basins.
- **Smooth Yet Tricky Surfaces** (e.g., Rosenbrock, Beale) can still pose considerable difficulty when the gradient is very small near narrow valleys or curved ravines.


# Implementing an Optimization Method (5 Points)

Now that we've discussed the general structure of our optimization function, let's explore how to implement a specific optimization method. In this section, we'll walk through the steps required to define an update rule for an optimizer. This involves:

1. **Computing the Loss and its Derivatives:**  
   We evaluate the loss function for the current parameters, compute the gradient, and optionally calculate the Hessian if using a second-order method.

2. **Applying the Update Rule:**  
   Based on the computed derivatives, we update the parameters using the specific logic of our chosen optimization method (e.g., adjusting the parameters using SGD, Adam, or Newton's method).

3. **Maintaining State:**  
   For methods that require momentum or other historical data, we update and maintain a state dictionary that carries this information from one iteration to the next.

By following these steps, we can iteratively update the parameters to minimize the loss function and analyze the optimization trajectory. Let's move on to see this process in action.

# General Optimization Function Explanation

The `optimize` function provides a unified framework for iterative optimization methods. Whether using first-order (gradient-based) or second-order (Hessian-based) methods, the overall process remains similar: we start with an initial guess for the parameters and then iteratively update these parameters in order to minimize a given loss function.

## How It Works

1. **Initialization:**
   - **Parameters:**  
     The function receives an initial parameter tensor (`initial_params`). This tensor is cloned and set to require gradients so that we can compute derivatives with respect to it.
   - **State Storage:**  
     An empty dictionary `state` is initialized to store any additional information required by the update method (e.g., momentum for Adam, or curvature approximations for second-order methods).
   - **Path Recording:**  
     The initial parameters are stored in a list called `path`. This list will hold the parameter values after each iteration, allowing us to visualize the optimization trajectory later.

2. **Iterative Update Process:**
   - For each epoch (i.e., optimization step), the following steps occur:
     1. **Loss Evaluation:**  
        The loss function `loss_func` is evaluated at the current parameter values to determine how well the current parameters perform.
     2. **Gradient Computation:**  
        The gradient (first derivative) of the loss with respect to the parameters is computed using `loss.backward()`.  
        - For **first-order methods**, only the gradient is used.
        - For **second-order methods**, additional curvature information is needed.
     3. **Hessian Computation (Optional):**  
        If the `second_order` flag is set to `True`, the Hessian (matrix of second derivatives) is computed using PyTorch's `torch.autograd.functional.hessian`.  
        This Hessian provides insight into the curvature of the loss surface and is useful for methods like Newton's method.
     4. **Parameter Update:**  
        The update function `update_func` is then called:
        - For **first-order methods** (e.g., SGD, Adam):  
          It is called with the current parameters, their gradient, the state, and hyperparameters.
        - For **second-order methods** (e.g., Newton’s Method, L-BFGS):  
          The computed Hessian is also passed.
        This function returns updated parameter values and an updated state.
     5. **Re-enable Gradient Tracking:**  
        After the update, gradients are cleared and re-enabled for the next iteration.
     6. **Path Recording:**  
        The new parameters are added to the `path` list.

3. **Return Value:**
   - After completing all epochs, the function returns the `path` — a list of parameter values at each iteration. This path can later be used to visualize the optimization trajectory (for example, by plotting the loss versus epochs or overlaying the path on the loss surface).

## Common Underlying Principles

- **Unified Structure:**  
  All optimization methods follow a similar iterative procedure: compute the loss, compute the gradient (and optionally the Hessian), update the parameters, and record the progress.
  
- **First-Order vs. Second-Order Methods:**  
  - *First-Order Methods* use only the gradient information. Their update rule typically has the form:  
  $$
    w_{t+1} = w_t - \eta \nabla f(w_t)
  $$
  - *Second-Order Methods* also incorporate curvature information (via the Hessian) to adjust the step direction and size:  
    $$ w_{t+1} = w_t - \left( H(w_t) + \epsilon I \right)^{-1} \nabla f(w_t) $$
  The `second_order` flag in our function determines whether the Hessian is computed and passed to the update function.

- **State Management:**  
  Many advanced optimizers (like Adam, RMSprop, or L-BFGS) maintain a state that includes historical information (e.g., moving averages of gradients, momentum, or curvature approximations). This state is updated at each iteration and passed to the update function to help guide the parameter updates.

In summary, the `optimize` function abstracts the common iterative process of optimization, allowing us to plug in different update functions (each corresponding to a different optimization method) and compare their performance on the same problem. This modular approach makes it straightforward to experiment with a wide range of optimization algorithms.


In [ ]:
import torch

def optimize(loss_func, update_func, initial_params, epochs, hyperparams, second_order=False):
    """
    General optimization function supporting both first-order (gradient-based)
    and second-order (Hessian-based) optimization methods.

    Parameters:
        - loss_func: Function to minimize. Accepts parameters and returns loss.
        - update_func: Function that updates parameters.
                       Signature (first-order): (params, grad, state, hyperparams) -> (new_params, new_state)
                       Signature (second-order): (params, grad, state, hyperparams, hessian) -> (new_params, new_state)
        - initial_params: Torch tensor representing the initial parameter values.
        - epochs: Number of optimization steps.
        - hyperparams: Dictionary of hyperparameters.
        - second_order: Boolean flag indicating if second-order derivatives (Hessian) are needed.

    Returns:
        - path: List of parameter values at each step.
    """
    params = initial_params.clone().detach().requires_grad_(True)
    state = {}  # Store state (e.g., momentum, Hessian approximations, etc.)
    path = [params.clone().detach().numpy()]  # Store initial point

    for epoch in range(epochs):
        loss = loss_func(params)  # Compute loss
        loss.backward()  # Compute gradient

        with torch.no_grad():  # Disable gradient tracking during updates
            grad = params.grad.clone()
            params.grad.zero_()
            
            if second_order:
                # TODO: Compute the Hessian matrix using torch.autograd.functional.hessian
                hessian = torch.autograd.functional.hessian(loss_func, params)

                # TODO: Update parameters using the update_func with second-order information
                params, state = update_func(params, grad, state, hyperparams, hessian)
            else:
                # TODO: Update parameters using the update_func with first-order information
                params, state = update_func(params, grad, state, hyperparams)

            params.requires_grad_(True)  # Re-enable gradient computation

        path.append(params.clone().detach().numpy())  # Store new point

    return path


# Optimization Demonstration Functions

Next, we have a set of functions that display the optimization process by running it on all the functions. You may edit them if needed for better visualization, but preferably, do not modify these two sections.


In [ ]:
def optimize_and_plot(loss_func, update_func, initial_params, epochs, hyperparams, x_domain, y_domain=None, title="Optimization" ,second_order=False):
    """
    Perform optimization and automatically visualize the results for 1D or 2D functions.

    Parameters:
      - loss_func: Function to minimize. It accepts parameters and returns a scalar loss.
      - update_func: Function that updates parameters. Signature: (params, grad, state, hyperparams) -> (new_params, new_state)
      - initial_params: Torch tensor representing the initial parameter values.
      - epochs: Number of optimization steps.
      - hyperparams: Dictionary of hyperparameters for the update function.
      - x_domain: Tuple (xmin, xmax) defining the domain for plotting.
      - y_domain: Tuple (ymin, ymax) defining the domain for 2D functions (set to None for 1D).
      - title: Title for the plots.

    Returns:
      - path: The list of parameter values at each epoch (for further processing if desired).
    """
    # Run the optimization (assumes a separate "optimize" function exists).
    path = optimize(loss_func, update_func, initial_params, epochs, hyperparams , second_order=second_order)
    path_np = np.array(path)  # Convert list of torch tensors (or arrays) to a NumPy array.
    initial_np = initial_params.detach().numpy()  # Get initial point as a NumPy array.

    # Check dimensionality: if initial_params has shape (1, ...) then it's 1D; if (2, ...) then 2D.
    if initial_params.shape[0] == 1:
        # 1D case: Pass initial_x and path.
        plot_1d_function_torch(
            func=loss_func,
            domain=x_domain,
            global_x=path_np[-1][0],
            title=title,
            initial_x=initial_np[0],
            path=path_np
        )
        plot_loss_vs_epoch(loss_func, path_np)
    elif initial_params.shape[0] == 2:
        # 2D case: Pass initial_point as a tuple and the full path.
        plot_2d_contour_torch(
            func=loss_func,
            x_domain=x_domain,
            y_domain=y_domain,
            global_point=(path_np[-1][0], path_np[-1][1]),
            title=title,
            initial_point=(initial_np[0], initial_np[1]),
            path=path_np
        )
        plot_interactive_3d_torch(
            func=loss_func,
            x_domain=x_domain,
            y_domain=y_domain,
            global_point=(path_np[-1][0], path_np[-1][1]),
            title=title,
            initial_point=(initial_np[0], initial_np[1]),
            path=path_np
        )
        plot_loss_vs_epoch(loss_func, path_np)
    else:
        raise ValueError("Function must be either 1D or 2D.")

    return


In [ ]:
def run_all_optimizations(update_func, hyperparams, epochs,second_order=False):
    """
    Run optimization for all benchmark functions (both 1D and 2D) using the provided update function,
    hyperparameters, and epoch count. It calls `optimize_and_plot` to handle visualization.

    Parameters:
      - update_func: The update function to use (e.g., adam_update, sgd_update, etc.).
      - hyperparams: Dictionary of hyperparameters for the update function.
      - epochs: Number of optimization steps to perform.
    """
    # List of benchmark function configurations.
    configs = [
        # 1D Functions
        {
            "title": "1D Rastrigin Function Optimization",
            "loss_func": rastrigin_1d_torch,
            "initial_params": torch.tensor([3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": None
        },
        {
            "title": "1D Schwefel Function Optimization",
            "loss_func": schwefel_1d_torch,
            "initial_params": torch.tensor([100.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (0, 500),
            "y_domain": None
        },
        {
            "title": "1D Ill-Conditioned Convex Function Optimization",
            "loss_func": ill_conditioned_convex_1d_torch,
            "initial_params": torch.tensor([3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": None
        },
        # 2D Functions
        {
            "title": "2D Rosenbrock Function Optimization",
            "loss_func": rosenbrock_2d_torch,
            "initial_params": torch.tensor([-1.0, 1.5], dtype=torch.float32, requires_grad=True),
            "x_domain": (-2, 2),
            "y_domain": (-1, 3)
        },
        {
            "title": "2D Rastrigin Function Optimization",
            "loss_func": rastrigin_2d_torch,
            "initial_params": torch.tensor([3.0, 3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": (-5, 5)
        },
        {
            "title": "2D Ackley Function Optimization",
            "loss_func": ackley_2d_torch,
            "initial_params": torch.tensor([3.0, 3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": (-5, 5)
        },
        {
            "title": "2D Beale Function Optimization",
            "loss_func": beale_2d_torch,
            "initial_params": torch.tensor([-3.0, 3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-4.5, 4.5),
            "y_domain": (-4.5, 4.5)
        },
        {
            "title": "2D Eggholder Function Optimization",
            "loss_func": eggholder_2d_torch,
            "initial_params": torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-512, 512),
            "y_domain": (-512, 512)
        },
        {
            "title": "2D Ill-Conditioned Convex Function Optimization",
            "loss_func": ill_conditioned_convex_2d_torch,
            "initial_params": torch.tensor([4.0, -3.0], dtype=torch.float32, requires_grad=True),
            "x_domain": (-5, 5),
            "y_domain": (-5, 5)
        }
    ]

    # Iterate over each function and call optimize_and_plot
    for config in configs:
        print("Optimizing:", config["title"])

        optimize_and_plot(
            loss_func=config["loss_func"],
            update_func=update_func,
            initial_params=config["initial_params"],
            epochs=epochs,
            hyperparams=hyperparams,
            x_domain=config["x_domain"],
            y_domain=config["y_domain"],
            title=config["title"],
            second_order=second_order
        )

        print("-" * 60)


# Implementation of Optimization Algorithms

Now that the optimization functions have been implemented, in this section we will implement various optimization algorithms. Some parts of these algorithms need to be implemented by you, while other parts have already been provided.

For each algorithm, experiment with different parameters and fine-tune them to achieve the best possible performance. The selection and tuning of these parameters are part of your grade, and the number of times you experiment with various settings is a crucial component of this exercise.

After implementing each algorithm, please submit a report detailing your observations and findings. Make sure to document your parameter tuning process and the performance outcomes for each algorithm.


# Stochastic Gradient Descent (SGD)(5 Points)

**Mathematical Formula:**

$$
w_{t+1} = w_t - \eta \nabla f(w_t)
$$

where:
- \\(w_t\\) is the parameter vector at iteration \\(t\\),
- \\(\eta\\) is the learning rate, and
- \\(\nabla f(w_t)\\) is the gradient of the loss function evaluated at \\(w_t\\).

**Explanation:**

Stochastic Gradient Descent (SGD) is a fundamental first-order optimization method that updates the parameters in the opposite direction of the gradient. At each iteration, the update rule moves the parameters by a small step proportional to the gradient and the learning rate. This method is widely used because of its simplicity and efficiency, particularly when dealing with large datasets. However, its performance heavily depends on the proper tuning of the learning rate; if set too high, the algorithm might overshoot minima, and if too low, convergence can be very slow.


In [ ]:
def sgd_update(params, grad, state, hyperparams, **kwargs):
    """
    SGD update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to the parameters.
      - state: Dictionary to store any state (unused in SGD).
      - hyperparams: Dictionary of hyperparameters. Must contain "lr" (learning rate).
      - **kwargs: Additional arguments (ignored for SGD).

    Returns:
      - new_params: Updated parameters.
      - state: Unchanged state (empty dictionary).
    """
    lr = hyperparams["lr"]  # Get the learning rate from hyperparameters.

    # TODO: Implement the standard SGD update rule
    new_params = params - lr * grad

    return new_params, state


In [ ]:
# Define hyperparameters for SGD (adjust the learning rate as needed).
sgd_hyperparams = {"lr": 0.001}

# Run optimizations for all benchmark functions using SGD.
run_all_optimizations(update_func=sgd_update, hyperparams=sgd_hyperparams, epochs=100)

# SGD Hyperparameter Analysis(5 Points)

For the SGD optimizer, we have defined the initial hyperparameters as follows:

    sgd_hyperparams = {"lr": 0.01}

In this section, you are required to test various learning rates and analyze how they affect the optimizer's performance. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
After experimenting with different values, we tested the following learning rates for SGD:

| Learning Rate | Observation |
|---------------|-------------------------------------- |
| 0.001         | Very slow convergence, but much better convergence. |
| 0.01          | Moderate speed, stable in most cases but diverges in complex 2D ones. |
| 0.05          | Faster initial convergence, but overshoots, oscillated and diverges in most cases. |
| 0.1           | Similar to 0.05.                |
| 0.5           | Almost always oscillates, diverges or fails to converge.                          |

2. **Performance Analysis:**
   - **Convergence Behavior**: 
      - **Lower LRs (0.001, 0.01)**: Showed more stable behavior and slower convergence. On simpler 1D functions, they converged successfully, although 0.001 often got stuck or barely moved after a certain point and 0.01 overshooting on some more complex functions.
      - **Mid-range LR (0.05)**: Helped the optimizer move quickly at first but caused oscillations or overshooting on functions like the 2D Rosenbrock or Rastrigin.
      - **Higher LRs (0.1, 0.5)**: Led to unstable behavior in most 2D tests, often diverging or looping indefinitely without settling.
   
   - **Unstable/Overshooting**: 
      - Learning rates of 0.05 and above often caused the optimizer to jump past minima or spiral outwards, especially noticeable in non-convex surfaces with narrow valleys.
      
   - **Getting Stuck in Local Minima**:
      - At 0.001, while it doesn’t overshoot, it moved so slowly that on functions with many local minima (e.g., 2D Rastrigin, Beale), it often stayed in a suboptimal region, effectively “stuck” there due to tiny gradient updates.

3. **Optimal Learning Rate:**
   - **Best Balance**: From the experiments, **0.001** provided the best overall trade-off between speed and stability. It generally avoided the severe overshooting seen with higher values and converged as fast as 0.01.

4. **Trade-offs and Observations:**
   - **Higher Learning Rates** (≥ 0.05): 
      - **Pros**: Faster initial descent on some simpler functions.  
      - **Cons**: High risk of overshooting or failing to converge entirely, especially on ill-conditioned or highly non-convex surfaces.
   - **Lower Learning Rates** (≤ 0.01): 
      - **Pros**: More stable updates, less oscillation.  
      - **Cons**: Very slow progress, increased likelihood of stagnation in local minima.
   - **Emerging Pattern**: Most of the “easier” 1D functions tolerated a broad range of learning rates. The more complex 2D benchmarks (Rosenbrock, Beale, Rastrigin) displayed significant sensitivity and tended to diverge or oscillate unless the learning rate was moderate.


5. **Recommendations:**
   - We should definitely start with lower values and then increase them based on the amount of oscillations, overshootings and convergence speed.

Make sure your report includes detailed observations, a comparative analysis, and a summary of all the learning rates tested. Your analysis should clearly illustrate how the choice of learning rate influences the performance of the SGD optimizer.


# SGD with Momentum (5 Points)

**Mathematical Formulation:**

The SGD with Momentum update is given by:

$$
v_t = \beta v_{t-1} + \eta \nabla f(w_t)
$$

$$
w_{t+1} = w_t - v_t
$$

where:  
- \\(w_t\\) is the parameter vector at iteration \\(t\\),  
- \\(v_t\\) is the velocity (accumulated momentum) at iteration \\(t\\),  
- \\(\eta\\) is the learning rate, and  
- \\(\beta\\) is the momentum coefficient (typically between 0 and 1).

**Explanation:**

SGD with Momentum introduces an additional term \\(v_t\\) that accumulates a running average of past gradients. This helps dampen oscillations and accelerates convergence, especially in scenarios where the gradient direction is consistent. However, careful tuning of both the learning rate \\(\eta\\) and the momentum coefficient \\(\beta\\) is necessary for optimal performance.


In [ ]:
def momentum_update(params, grad, state, hyperparams, **kwargs):
    """
    SGD with Momentum update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to the parameters.
      - state: Dictionary to store state (e.g., velocity). Initially, this can be empty.
      - hyperparams: Dictionary of hyperparameters. Must contain:
          "lr": learning rate, and
          "momentum": momentum coefficient.
      - **kwargs: Additional arguments (not used here).

    Returns:
      - new_params: Updated parameters.
      - state: Updated state dictionary (contains the velocity).
    """
    lr = hyperparams["lr"]
    momentum = hyperparams["momentum"]

    if "v" not in state:
        # TODO: Initialize the velocity tensor with zeros, having the same shape as params
        state["v"] = torch.zeros_like(params)

    # TODO: Implement the momentum update rule: v_t = momentum * v_{t-1} + lr * grad
    state["v"] = momentum * state["v"] + lr * grad

    # TODO: Update parameters using the velocity term
    new_params = params - state["v"]

    return new_params, state


In [ ]:
# Define hyperparameters for SGD with Momentum.
momentum_hyperparams = {
    "lr": 0.001,       # learning rate
    "momentum": 0.9    # momentum coefficient
}

# Run optimizations for all benchmark functions using SGD with Momentum.
run_all_optimizations(update_func=momentum_update, hyperparams=momentum_hyperparams, epochs=100)


# Momentum Hyperparameter Analysis(5 Points)

For the Momentum optimizer, the hyperparameters are defined as follows:




```python
momentum_hyperparams = {
    "lr": 0.001,       # learning rate
    "momentum": 0.9    # momentum coefficient
}
```




In this section, you are required to test various values for both the learning rate and the momentum coefficient, and analyze their impact on the optimizer's performance. Please answer the following questions based on your experiments:

1. Parameter Variation:
   Below is a summary table of the main combinations:

| Learning Rate | Momentum | Observations                                               |
|---------------|----------|------------------------------------------------------------|
| 0.0001        | 0.5      | Very slow but stable; occasionally stuck in local minima.  |
| 0.0001        | 0.9      | Slow convergence; fewer oscillations.                      |
| 0.0001        | 0.95     | Slight speed-up, still quite slow overall.                |
| 0.0001        | 0.99     | Slower, but smoother updates; might not escape some minima.|
| 0.001         | 0.5      | Moderately stable; can be a bit slower on rugged landscapes.|
| 0.001         | 0.9      | Good overall performance; decent speed + stability.        |
| 0.001         | 0.95     | Faster initial progress; occasional overshoot on complex surfaces. |
| 0.001         | 0.99     | Can converge quickly if not trapped; some oscillation possible.|
| 0.01          | 0.5      | Faster convergence in 1D tests; can overshoot in 2D.       |
| 0.01          | 0.9      | Tends to overshoot on ill-conditioned or narrow-valley functions. |
| 0.01          | 0.95     | Rapid movement initially, leading to instability on functions like Beale. |
| 0.01          | 0.99     | Frequent strong oscillations; prone to divergence in complex landscapes. |
| 0.1           | 0.9      | Almost always diverges quickly on tricky functions.        |
| 0.1           | 0.95     | Very unstable; partial or complete divergence.            |
| 0.1           | 0.99     | Highly unstable in most cases.                             |

2. Performance Analysis:
   - **Learning Rate Effects**:
      - **Lower LRs (≤ 0.0001 or 0.001)**: 
         - Yielded stable updates and less oscillation, but sometimes converged too slowly (or got stuck in local minima).
         - Often needed more epochs to escape rugged areas.
      - **Mid to High LRs (≥ 0.01)**:
         - Offered faster initial descent on simpler 1D functions.
         - Caused overshooting or even divergence on complex 2D functions like Rosenbrock, Beale, and Rastrigin.
   - **Momentum Coefficient Variations**:
      - **Lower Momentum (0.5)**:
         - Reduced the “boost” effect, so updates remained smaller and more controlled.
         - Convergence was steadier but often slower, especially in wide plateaus.
      - **Higher Momentum (≥ 0.9)**:
         - Accelerated progress along consistent gradient directions, often improving initial convergence speed.
         - Could cause oscillations or overshooting in narrow valleys or sharply curved regions (e.g., 2D Rosenbrock).
         - Very high momentum (0.99) risked buildup of large “velocity” terms, making the optimizer difficult to control on difficult surfaces.
   - **Faster Convergence vs. Instability**:
      - **Combinations with LR ~ 0.001 and momentum ~ 0.9 or 0.95** struck a good balance on most benchmarks, converging relatively quickly without severe divergence.
      - **High LR + high momentum** often led to extreme oscillation or divergence, especially evident in Beale and ill-conditioned functions.


3. Optimal Parameter Combination:
   - **Best Combination**: Across the tests, **LR = 0.001** and **momentum = 0.9** provided the most consistent results—good convergence speed on simpler landscapes while staying stable on more complex ones.
   - **Why Optimal?**:
      - This parameter set typically avoided the slow crawling of extremely low LR or the instability of higher LR values.
      - The momentum of 0.9 accelerated the optimizer sufficiently without causing uncontrollable oscillations in most cases.


4. Trade-offs and Observations:
   - **Higher Momentum**:
      - **Pros**: Faster descent along well-defined gradient directions; can “push through” minor local traps.
      - **Cons**: More prone to overshoot in curved valleys; might cycle or diverge if combined with a high LR.
   - **Lower Momentum**:
      - **Pros**: More controlled, stable updates; less risk of large oscillations.
      - **Cons**: Slower progress, especially on functions with broad plateaus or shallow gradients.
   - **Learning Rate Trends**:
      - Big steps (≥ 0.01) can boost speed on simpler functions but risk divergence on complex ones.
      - Tiny steps (≤ 0.0001) generally avoid overshooting but converge very slowly or stall in local minima.


5. Recommendations:
   - Quite similar to the previous one, we should start with lower learning rates, then we can change the momentum in a way to minimize the oscillations and divergences. Then we can slowly start to use higher learning rates.

Ensure your report includes detailed observations, a comparative analysis, and a summary of all the hyperparameter values tested. Your analysis should clearly illustrate how the choice of learning rate and momentum coefficient influences the performance of the Momentum optimizer.


# Nesterov Accelerated Gradient (NAG)

**Mathematical Formulation:**

The Nesterov Accelerated Gradient update is given by:

$$
v_t = \beta \, v_{t-1} + \eta \, \nabla f(w_t)
$$

$$
w_{t+1} = w_t - \beta \, v_{t-1} - \eta \, \nabla f(w_t)
$$

where:
- \\( w_t \\) is the parameter vector at iteration \\( t \\),
- \\( v_t \\) is the velocity (accumulated momentum) at iteration \\( t \\),
- \\( \eta \\) is the learning rate, and
- \\( \beta \\) is the momentum coefficient (typically a value between 0 and 1).

**Explanation:**

Nesterov Accelerated Gradient (NAG) is a variant of SGD with momentum that computes the gradient at a "lookahead" position. Instead of computing the gradient at the current parameter \\( w_t \\) only, NAG uses the accumulated momentum from previous iterations to “look ahead” before updating the parameters. The update can be interpreted as:

1. **Compute the momentum update:**  
   \\( v_t = \beta \, v_{t-1} + \eta \, \nabla f(w_t) \\)
2. **Update the parameters using the previous momentum:**  
   \\( w_{t+1} = w_t - \beta \, v_{t-1} - \eta \, \nabla f(w_t) \\)

This "lookahead" strategy often leads to faster convergence compared to standard momentum because it anticipates the effect of momentum and adjusts the update accordingly.

*Note:* In our implementation, we assume that we cannot recompute the gradient at the lookahead position (for simplicity) so we approximate NAG with the following update:

$$
v_{\text{prev}} = v_{t-1} \quad,\quad v_t = \beta \, v_{t-1} + \eta \, \nabla f(w_t)
$$

$$
w_{t+1} = w_t - \beta \, v_{\text{prev}} - \eta \, \nabla f(w_t)
$$

This formulation uses the previous momentum term directly for the "lookahead" component.


In [ ]:
def nag_update(params, grad, state, hyperparams, **kwargs):
    """
    Nesterov Accelerated Gradient (NAG) update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params.
      - state: Dictionary to store the momentum (velocity). Initially empty.
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "lr": learning rate, and
            "momentum": momentum coefficient.
      - **kwargs: Additional arguments (ignored for NAG).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary containing the velocity.
    """
    lr = hyperparams["lr"]
    momentum = hyperparams["momentum"]

    # Initialize the velocity if not already in state.
    if "v" not in state:
        state["v"] = torch.zeros_like(params)

    # Save the previous velocity for the lookahead.
    v_prev = state["v"].clone()
    # Update the velocity.
    state["v"] = momentum * state["v"] + lr * grad
    # Update the parameters using the lookahead formula.
    new_params = params - momentum * v_prev - lr * grad
    return new_params, state


In [ ]:
# Define hyperparameters for Nesterov Accelerated Gradient.
nag_hyperparams = {
    "lr": 0.001,        # Learning rate
    "momentum": 0.9     # Momentum coefficient
}

# Run optimizations for all benchmark functions using NAG.
run_all_optimizations(update_func=nag_update, hyperparams=nag_hyperparams, epochs=100)


# NAG Hyperparameter Analysis(5 Points)

For the Nesterov Accelerated Gradient (NAG) optimizer, we have defined the initial hyperparameters as follows:

    nag_hyperparams = {
        "lr": 0.001,        # Learning rate
        "momentum": 0.9     # Momentum coefficient
    }

In this section, you are required to test various learning rates and momentum values to analyze how they affect the performance of NAG. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - Similar to the momentum-based experiments, we explored several learning rates and momentum coefficients:

| Learning Rate | Momentum | Observations                                               |
|---------------|----------|------------------------------------------------------------|
| 0.0001        | 0.5      | Very slow, stable; minimal overshoot, but may stall.       |
| 0.0001        | 0.9      | Slower but smoother, especially on non-convex surfaces.    |
| 0.0001        | 0.95     | Still very slow, though it helps push through minor dips.  |
| 0.0001        | 0.99     | Smooth updates; might remain stuck if gradients are tiny.  |
| 0.001         | 0.5      | Moderate speed; little risk of major oscillations.         |
| 0.001         | 0.9      | Generally stable, relatively fast convergence.            |
| 0.001         | 0.95     | Faster initial movement; occasional overshoot on complex landscapes. |
| 0.001         | 0.99     | Can speed up but risks bigger swings in sharp valleys.     |
| 0.01          | 0.5      | Noticeably faster on simpler functions; minor risk of overshoot.  |
| 0.01          | 0.9      | Potentially unstable on highly curved surfaces (e.g., Beale).  |
| 0.01          | 0.95     | Fast initial progress; can diverge or oscillate if gradients are steep. |
| 0.01          | 0.99     | Strong, persistent oscillations in complex regions.        |
| 0.1           | 0.9      | Often diverges in most 2D non-convex functions.           |
| 0.1           | 0.95     | Very unstable; large swings in function value.            |
| 0.1           | 0.99     | Extremely prone to divergence.                             |

2. **Performance Analysis:**
   - **Impact of Learning Rate**:
      - **Lower LRs (≤ 0.0001 or 0.001)**: More controlled progress, fewer oscillations or divergences, but can be slow on functions with wide plateaus or many local minima.
      - **Higher LRs (≥ 0.01)**: Much quicker descent initially, but prone to overshoot or diverge in narrow valleys and ill-conditioned functions (e.g., Rosenbrock).

   - **Momentum Coefficient Variations**:
      - **Lower Momentum (0.5)**: Provides more gradual acceleration, resulting in steadier but slower convergence.  
      - **Higher Momentum (≥ 0.9)**: Speeds up progress in consistent gradient directions. However, on tricky 2D functions, it can overshoot or oscillate if combined with a larger LR.
      - **NAG’s “Lookahead”**: Tends to reduce some overshooting compared to standard momentum but can still oscillate if LR and momentum are both set too high.

   - **Instabilities and Local Minima**:
      - Overshooting and partial divergence were most common with LR≥0.01 and momentum≥0.9, especially on Beale and other multi-modal surfaces.
      - Extremely low LR (e.g., 0.0001) occasionally led to the optimizer sitting in local minima too long, particularly on functions with rugged landscapes.

3. **Optimal Parameters:**
   - **Best Balance**: **Learning Rate = 0.001** and **Momentum = 0.9** emerged again as a solid compromise.  
   - **Why Optimal?**: 
      - Consistently avoided severe divergence or chaotic oscillations on the more complex tests.
      - Provided reasonably fast convergence without requiring excessive tuning across different functions.

4. **Trade-offs and Observations:**
   - **Higher Momentum**:
      - **Pros**: Accelerates the search, helps skip over small bumps.  
      - **Cons**: If combined with a moderately high LR, it can produce strong oscillations or divergence.
   - **Lower Momentum**:
      - **Pros**: More stable, fewer swings.  
      - **Cons**: Slower to leave plateaus and not as efficient in deeper valleys.
   - **Key Trend**: NAG’s lookahead generally helps reduce some overshooting issues seen in basic momentum, but extreme parameter values (high LR + high momentum) still risk instability.

5. **Recommendations:**
   - Similar to SGD + Momentum

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the NAG optimizer.


# Adagrad (Adaptive Gradient)

**Mathematical Formulation:**

Adagrad adapts the learning rate for each parameter individually by accumulating the squared gradients. Its update equations are:

$$
G_t = G_{t-1} + \left(\nabla f(w_t)\right)^2
$$

$$
w_{t+1} = w_t - \frac{\eta}{\sqrt{G_t} + \epsilon} \, \nabla f(w_t)
$$

where:  
- \\(w_t\\) is the parameter vector at iteration \\(t\\),  
- \\(G_t\\) is the accumulation of squared gradients (applied elementwise),  
- \\(\eta\\) is the learning rate, and  
- $\epsilon$ is a small constant added for numerical stability.

**Explanation:**

Adagrad adjusts the learning rate based on the history of gradients. Parameters with high gradients accumulate a larger \\(G_t\\) and thus receive a smaller effective learning rate, while parameters with small or sparse gradients receive larger updates. This makes Adagrad particularly useful for problems with sparse features, but note that its learning rate can decay too aggressively over time.


In [ ]:
def adagrad_update(params, grad, state, hyperparams, **kwargs):
    """
    Adagrad update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to the parameters.
      - state: Dictionary to store state (e.g., accumulated squared gradients).
      - hyperparams: Dictionary of hyperparameters. Must contain:
          "lr": learning rate, and
          "eps": a small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for Adagrad).

    Returns:
      - new_params: Updated parameters.
      - state: Updated state dictionary.
    """
    lr = hyperparams["lr"]
    eps = hyperparams["eps"]

    if "G" not in state:
        state["G"] = torch.zeros_like(params)

    # Accumulate squared gradients.
    state["G"] += grad**2

    # Perform the Adagrad update.
    new_params = params - lr * grad / (torch.sqrt(state["G"]) + eps)
    return new_params, state


In [ ]:
# Define hyperparameters for Adagrad.
adagrad_hyperparams = {
    "lr": 0.01,   # Learning rate
    "eps": 1e-7   # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Adagrad.
run_all_optimizations(update_func=adagrad_update, hyperparams=adagrad_hyperparams, epochs=100)


# Adagrad Hyperparameter Analysis(5 Points)

For the Adagrad optimizer, we have defined the initial hyperparameters as follows:

    adagrad_hyperparams = {
        "lr": 0.01,   # Learning rate
        "eps": 1e-8   # Small constant for numerical stability
    }

In this section, you are required to test various learning rates and epsilon values to analyze how they affect the performance of Adagrad. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - I tested a range of learning rates and epsilon ($\epsilon$) values:

| Learning Rate | Epsilon | Observations                                                             |
|---------------|---------|--------------------------------------------------------------------------|
| 0.001         | 1e-8    | Very slow but stable; can stall on rugged or multi-modal functions.      |
| 0.001         | 1e-7    | Slightly faster than with 1e-8, still rather conservative updates.       |
| 0.001         | 1e-6    | Generally stable; no serious overshoot but might not fully converge in 100 epochs. |
| 0.001         | 1e-4    | Improved speed, occasional risk of partial overshoot on certain 2D tests.|
| 0.01          | 1e-8    | Good balance of speed and stability on most functions.                   |
| 0.01          | 1e-7    | Very similar performance to 1e-8, converges slightly faster in some cases.|
| 0.01          | 1e-6    | Faster convergence; minor oscillations on complex landscapes.            |
| 0.01          | 1e-4    | Risk of overshoot in non-convex or ill-conditioned surfaces.             |
| 0.1           | 1e-8    | High risk of oscillations or divergence, especially on 2D complex functions. |
| 0.1           | 1e-7    | Similar to above; frequent overshooting.                                 |
| 0.1           | 1e-6    | Rapid updates; can converge quickly on easy functions, diverge on tricky ones.|
| 0.1           | 1e-4    | Most prone to instability or partial divergence.                          |


2. **Performance Analysis:**
   - **Learning Rate Effects**:
      - **Low LR (0.001)**: Very stable updates, but slow convergence, especially on functions with broad flat regions or numerous local minima.
      - **Moderate LR (0.01)**: Balanced speed and stability, generally converging on most functions without extreme oscillations.
      - **High LR (0.1)**: Can cause overshooting or even divergence, especially on highly non-convex or ill-conditioned surfaces.

   - **Epsilon ($\epsilon$) Variations**:
      - **Very Small $\epsilon$ (1e-8)**: Provides the original flavor of Adagrad with minimal numerical influence, typically stable but can be slow if gradients become very small.
      - **Larger $\epsilon$ (≥1e-6)**: Can help keep updates from shrinking too fast in later stages, leading to faster movement but occasionally introducing minor oscillations.

   - **Unstable Behavior / Local Minima**:
      - Some overshoot at **LR=0.1** combined with moderate-to-large $\epsilon$ values, especially for complicated surfaces like 2D Rastrigin or Eggholder.
      - At **very low LR (0.001)**, the optimizer can essentially plateau in local minima if the accumulated gradient scaling quickly shrinks step sizes.


3. **Optimal Parameters:**
   - **Best Combination**: A **learning rate around 0.01** with an **$\epsilon$ of about 1e-7 or 1e-8** often provided the best trade-off between convergence speed and stability across various benchmarks.
   - **Reasoning**: 
   - At LR=0.01, Adagrad updates aren’t so large that they overshoot, yet they remain large enough for adequate exploration.
   - A slightly smaller $\epsilon$ (1e-8 or 1e-7) ensures that the step-size reduction mechanism remains significant without causing too much numerical dampening.


4. **Trade-offs and Observations:**
   - **Lower $\epsilon$**:
      - **Pros**: Keeps Adagrad’s adaptive mechanism intact for stronger diminishing steps over time, leading to stable convergence.
      - **Cons**: May reduce step sizes very quickly, slowing final convergence or getting stuck in local minima.
   - **Higher $\epsilon$**:
      - **Pros**: Helps maintain moderately sized updates longer, preventing the optimizer from stalling.
      - **Cons**: Can introduce small oscillations or overshooting on sharply curved functions.
   - **Learning Rate Trends**:
      - A moderate LR avoids both the “forever stuck” scenario of very small LR and the instability of excessively large LR.
      - On simpler 1D surfaces, even a slightly higher LR can be fine, but on more complex 2D problems, it can cause divergence.

5. **Recommendations:**
   - In simples terms, we can start with something like 0.01 as the learning rate and 1e-7 for the epsilon. Then can start changing the epsilon base on whether the models is stalling, overshooting, diverging, etc. Also slightly smaller learning rates are better for more complex function (to ensure that we won't overshoot or diverge).

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the Adagrad optimizer.


# RMSprop (Root Mean Square Propagation)

**Mathematical Formulation:**

RMSprop adapts the learning rate for each parameter by maintaining an exponentially decaying average of squared gradients. The update equations are:

$$
E[g^2]_t = \beta \, E[g^2]_{t-1} + (1 - \beta) \, \left(\nabla f(w_t)\right)^2
$$

$$
w_{t+1} = w_t - \frac{\eta}{\sqrt{E[g^2]_t} + \epsilon} \, \nabla f(w_t)
$$

where:
- \\(w_t\\) is the parameter vector at iteration \\(t\\),
- \\(E[g^2]_t\\) is the exponentially decaying average of squared gradients,
- \\(\eta\\) is the learning rate,
- \\(\beta\\) is the decay rate (typically around 0.9), and
- $\epsilon$ is a small constant added for numerical stability.

**Explanation:**

RMSprop is designed to overcome the aggressive decay of the learning rate seen in Adagrad by using an exponential decay factor \\(\beta\\) to forget old gradients. This helps to maintain a more stable and adaptive learning rate during training. RMSprop works well in practice on non-stationary problems and deep networks.


In [ ]:
def rmsprop_update(params, grad, state, hyperparams, **kwargs):
    """
    RMSprop update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to parameters.
      - state: Dictionary to store state (e.g., running average of squared gradients).
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "lr": learning rate,
            "beta": decay rate, and
            "eps": a small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for RMSprop).

    Returns:
      - new_params: Updated parameters.
      - state: Updated state dictionary.
    """
    lr = hyperparams["lr"]
    beta = hyperparams["beta"]
    eps = hyperparams["eps"]

    if "E" not in state:
        state["E"] = torch.zeros_like(params)

    # Update the running average of squared gradients.
    state["E"] = beta * state["E"] + (1 - beta) * grad**2

    # Compute the RMSprop update.
    new_params = params - lr * grad / (torch.sqrt(state["E"]) + eps)
    return new_params, state


In [ ]:
# Define hyperparameters for RMSprop.
rmsprop_hyperparams = {
    "lr": 0.001,    # Learning rate
    "beta": 0.9,    # Decay rate
    "eps": 1e-8     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using RMSprop.
run_all_optimizations(update_func=rmsprop_update, hyperparams=rmsprop_hyperparams, epochs=1000)


# RMSprop Hyperparameter Analysis(5 Points)

For the RMSprop optimizer, we have defined the initial hyperparameters as follows:

    rmsprop_hyperparams = {
        "lr": 0.001,    # Learning rate
        "beta": 0.9,    # Decay rate
        "eps": 1e-8     # Small constant for numerical stability
    }

In this section, you are required to test various learning rates, beta values, and epsilon values to analyze how they affect the performance of RMSprop. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - I experimented with a few ranges for each hyperparameter:

| Learning Rate | Beta  | Epsilon | Observations                                                              |
|---------------|-------|---------|---------------------------------------------------------------------------|
| 0.0001        | 0.9   | 1e-8    | Very slow, stable updates; might not reach deeper minima within 1000 epochs. |
| 0.0001        | 0.9   | 1e-6    | Slightly faster than 1e-8, still limited progress in complex landscapes.   |
| 0.0001        | 0.9   | 1e-4    | Gradually converges on simpler functions but stalls in multi-modal ones.  |
| 0.0001        | 0.99  | 1e-8    | Extremely slow adaptation; minimal oscillation, sometimes stuck in local minima. |
| 0.001         | 0.9   | 1e-8    | Balanced speed, moderate success on a variety of functions.               |
| 0.001         | 0.9   | 1e-6    | Slight oscillations on tricky 2D surfaces, still relatively stable.       |
| 0.001         | 0.9   | 1e-4    | Faster updates but can overshoot in steep regions.                        |
| 0.001         | 0.99  | 1e-8    | Very smooth updates, sometimes too slow on rugged functions.             |
| 0.001         | 0.99  | 1e-6    | Helpful in smoothing out noise but can converge slowly.                   |
| 0.01          | 0.9   | 1e-8    | Quick initial descent; prone to overshooting or oscillation on difficult functions. |
| 0.01          | 0.9   | 1e-6    | Faster adaptation, but can get unstable or stuck in local minima.         |
| 0.01          | 0.99  | 1e-8    | Large decay, slow parameter updates if gradients are not consistent.      |
| 0.01          | 0.99  | 1e-6    | High likelihood of overshooting and diverging on ill-conditioned surfaces.|


2. **Performance Analysis:**
   - **Learning Rate (LR)**:
      - **0.0001**: Very conservative; stable updates but insufficient movement in 1000 epochs for complex or multi-modal functions.
      - **0.001**: Struck a reasonable balance of speed and control, generally avoiding major oscillations while still exploring.
      - **0.01**: Tends to move quickly but frequently overshoots on rugged 2D benchmarks, sometimes failing to settle into global minima.

   - **Beta (Decay Rate)**:
      - **0.9**: Provides a moderate smoothing of gradient variance. Often yields quicker adaptation while keeping some agility to respond to new gradient information.
      - **0.99**: Stronger smoothing effect can slow changes significantly. While it reduces oscillations further, it can cause the optimizer to “creep” toward minima and struggle with highly non-convex terrains.

   - **Epsilon ($\epsilon$)**:
      - **1e-8**: Minimal baseline to prevent division by zero; stable but can cause very small updates if the squared gradient accumulates a lot.
      - **1e-6 and 1e-4**: Allows slightly larger effective step sizes when gradients are small, potentially improving convergence speed but sometimes introducing minor overshoots.

   - **Unstable or Slow Convergence**:
      - High LR (0.01) with high beta (0.99) was a frequent culprit for oscillation or partial divergence, especially on 2D Rastrigin or Eggholder.
      - Very low LR (0.0001) with high beta (0.99) caused extremely slow progress, sometimes trapping the optimizer in suboptimal minima.

3. **Optimal Parameters:**
   - **Best Combination**: **LR = 0.001, Beta = 0.9, Epsilon = 1e-8** generally worked well across various functions, offering stability with moderate convergence speed.
   - **Reasoning**: 
      - 0.001 is often a sweet spot for RMSprop, neither too large to overshoot nor too small to stall.
      - Beta=0.9 smooths out gradient variance without dragging updates too slowly.
      - 1e-8 ensures numerical stability without altering the adaptive update too aggressively.


4. **Trade-offs and Observations:**
   - **Higher Beta**:
      - **Pros**: More damping of rapid gradient changes, reducing oscillations.
      - **Cons**: Can slow adaptation significantly in rugged or shifting landscapes.
   - **Lower Beta**:
      - **Pros**: More responsive to changes in the gradient, faster in early iterations.
      - **Cons**: Might see slightly more oscillation or sensitivity to local gradient noise.
   - **Epsilon Variations**:
      - A larger $\epsilon$ (1e-6 or 1e-4) may help avoid extremely small step sizes late in training, but can introduce extra bounce in the updates.
   - **Patterns**:
      - RMSprop typically stabilizes more quickly than basic SGD or momentum, but may plateau, failing to fully optimize the most challenging multi-modal functions unless carefully tuned.

5. **Recommendations:**
   - First, you can start with something like, 0.001 as the learning rate, 0.9 for the beta and 1e-7 for the epsilon. Then you can increase or decrease the beta if you have oscillations or slow convergence speed.

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the RMSprop optimizer.


# Adam (Adaptive Moment Estimation)(10 Points)

**Mathematical Formulation:**

Adam computes two exponential moving averages of the gradients:

- **First moment (mean):**

$$
m_t = \beta_1 m_{t-1} + (1 - \beta_1) \nabla f(w_t)
$$

- **Second moment (uncentered variance):**

$$
v_t = \beta_2 v_{t-1} + (1 - \beta_2) \left(\nabla f(w_t)\right)^2
$$

Bias corrections are then applied:

$$
\hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
$$

Finally, the parameter update is given by:

$$
w_{t+1} = w_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \, \hat{m}_t
$$

where:
- \\(w_t\\) is the parameter vector at iteration \\(t\\),
- \\(\eta\\) is the learning rate,
- \\(\beta_1\\) and \\(\beta_2\\) are the exponential decay rates for the moment estimates, and
- $\epsilon$ is a small constant to avoid division by zero.

**Explanation:**

Adam combines ideas from momentum and RMSprop. By maintaining an exponentially decaying average of past gradients (first moment) and past squared gradients (second moment) along with bias correction, Adam adapts the learning rate for each parameter. This makes it especially suitable for problems with noisy or sparse gradients, as it works well "out of the box" with minimal tuning.


In [ ]:
def adam_update(params, grad, state, hyperparams, **kwargs):
    """
    Adam update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params.
      - state: Dictionary to store moving averages for first and second moments.
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "lr": learning rate,
            "beta1": decay rate for the first moment,
            "beta2": decay rate for the second moment,
            "eps": a small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for Adam).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary.
    """
    lr = hyperparams["lr"]
    beta1 = hyperparams["beta1"]
    beta2 = hyperparams["beta2"]
    eps = hyperparams["eps"]

    if "m" not in state:
        # TODO: Initialize first moment (m), second moment (v), and timestep (t)
        state["m"] = torch.zeros_like(params)
        state["v"] = torch.zeros_like(params)
        state["t"] = 0

    # TODO: Update timestep
    state["t"] += 1

    # TODO: Compute biased first moment estimate (m) and second moment estimate (v)
    state["m"] = beta1 * state["m"] + (1 - beta1) * grad
    state["v"] = beta2 * state["v"] + (1 - beta2) * grad ** 2

    # TODO: Compute bias-corrected first moment estimate (m_hat) and second moment estimate (v_hat)
    m_hat = state["m"] / (1 - beta1 ** state["t"])
    v_hat = state["v"] / (1 - beta2 ** state["t"])

    # TODO: Update parameters using Adam's rule
    new_params = params - (lr * m_hat) / (torch.sqrt(v_hat) + eps)

    return new_params, state


In [ ]:
# Define hyperparameters for Adam.
adam_hyperparams = {
    "lr": 0.001,      # Learning rate
    "beta1": 0.9,     # Decay rate for first moment
    "beta2": 0.999,   # Decay rate for second moment
    "eps": 1e-8       # Smoothing constant
}

# Run optimizations for all benchmark functions using Adam.
run_all_optimizations(update_func=adam_update, hyperparams=adam_hyperparams, epochs=1000)


# Adam Hyperparameter Analysis(5 Points)

For the Adam optimizer, we have defined the initial hyperparameters as follows:

    adam_hyperparams = {
        "lr": 0.001,      # Learning rate
        "beta1": 0.9,     # Decay rate for first moment
        "beta2": 0.999,   # Decay rate for second moment
        "eps": 1e-8       # Smoothing constant
    }

In this section, you are required to test various combinations of the learning rate, beta1, beta2, and epsilon values to analyze how they affect the performance of Adam. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - I experimented with a range of values for each hyperparameter:

| LR     | Beta1 | Beta2 | Epsilon | Observations                                                                     |
|--------|-------|-------|---------|----------------------------------------------------------------------------------|
| 0.0001 | 0.9   | 0.999 | 1e-8    | Very slow but stable; can plateau on rugged/multi-modal surfaces.               |
| 0.0001 | 0.9   | 0.999 | 1e-6    | Similar speed, slightly more bounce in updates.                                 |
| 0.0001 | 0.95  | 0.999 | 1e-8    | Converges steadily, but sometimes stalls before reaching global minima.         |
| 0.001  | 0.9   | 0.999 | 1e-8    | Good balance of speed and stability on most benchmarks.                         |
| 0.001  | 0.9   | 0.999 | 1e-6    | Slight oscillations on complex surfaces; still generally stable.                |
| 0.001  | 0.9   | 0.9   | 1e-8    | Faster adaptation but can overshoot on steep or ill-conditioned landscapes.     |
| 0.001  | 0.95  | 0.999 | 1e-8    | Momentum effect is stronger; typically stable with reduced noise, but can be slower on very complex tasks. |
| 0.01   | 0.9   | 0.999 | 1e-8    | Quick descent; prone to overshooting or oscillations in multi-modal functions.   |
| 0.01   | 0.9   | 0.999 | 1e-6    | Risk of divergence on particularly challenging 2D surfaces (e.g., Eggholder).    |
| 0.01   | 0.95  | 0.999 | 1e-8    | Rapid initial progress; can oscillate severely if gradients become large.        |


2. **Performance Analysis:**
   - **Impact of Learning Rate**:
      - **Lower LR (0.0001)**: Very cautious updates, reducing the chance of overshoot but often too slow for complex or large-scale problems within a limited epoch count.
      - **Moderate LR (0.001)**: Typically strikes a strong balance—fast enough to escape small local minima but not so large as to cause unstable swings.
      - **High LR (0.01)**: Faster initial descent; more vulnerable to oscillations or divergence in functions with sharp curvature or narrow valleys.

   - **Beta1**:
      - **0.9**: The classic default, offering a well-balanced momentum effect that smooths out gradients without retaining too much past information.
      - **0.95**: Increased momentum can sometimes accelerate progress along consistent directions but risks slower course correction when gradients shift abruptly.

   - **Beta2**:
      - **0.999**: A common default providing substantial smoothing of the second moment (variance). Effective at stabilizing updates but can adapt slowly to dramatic changes in gradient magnitude.
      - **0.9**: More responsive to sudden changes, though can lead to larger update swings if the gradient variance changes rapidly.

   - **Epsilon ($\epsilon$)**:
      - **1e-8**: The default minimal offset for stability; leads to more pronounced adaptive behavior in Adam.
      - **1e-6**: A higher epsilon may keep updates from shrinking too much in later stages, but can also add slight oscillations, especially when combined with a moderate/high learning rate.

   - **Unstable / Slow Convergence**:
      - High LR (0.01) + high momentum (Beta1=0.95) + small Beta2 (0.9) can result in strong oscillations or partial divergence.
      - Low LR (0.0001) with large Beta2 (0.999) often leads to extremely slow progress, sometimes not escaping local minima in time.


3. **Optimal Parameters:**
   - **Best Overall Combination**: **LR = 0.001, Beta1 = 0.9, Beta2 = 0.995, Epsilon = 1e-8** remains the most robust across diverse functions.
   - **Why?**:
      - This standard Adam setup typically converges relatively quickly for simpler surfaces and remains stable for more challenging ones.
      - Adjustments to Beta1 and Beta2 can help fine-tune performance but rarely outperform the baseline on all problems.

4. **Trade-offs and Observations:**
   - **Higher Beta1 (≥ 0.95)**: 
      - **Pros**: Stronger momentum can help skip minor local dips.  
      - **Cons**: Risk of slower course corrections if the gradient’s direction changes suddenly.
   - **Lower Beta2 (0.9 vs. 0.999)**:
      - **Pros**: More responsive to abrupt changes, quicker adaptation to new gradients.  
      - **Cons**: Updates can become noisier, possibly causing overshoots in complex landscapes.
   - **Learning Rate Trends**: 
      - Above 0.001, watch for overshoot; below 0.001, watch for underfitting or slow convergence.
   - **Epsilon Variations**:
      - Large epsilon helps keep updates from vanishing but can reduce Adam’s adaptive scaling precision, especially with moderate to high LR.


5. **Recommendations:**
   - One problem with ADAM is its number of parameters. You should use many different combinations of them in order to find the perfect one. But you can start with something like 0.0001 as the learning rate, 0.9 for beta1, 0.999 for beta2 and 1e-7 for the epsilon. Then we can change them base on the convergence speed, oscillations, smoothness and etc.

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the Adam optimizer.


# Newton’s Method(5 Points)

**Mathematical Formulation:**

Newton’s method uses second-order information (the Hessian) to update the parameters. The update rule is given by:

$$
w_{t+1} = w_t - H(w_t)^{-1} \nabla f(w_t)
$$

where:
- \\( w_t \\) is the parameter vector at iteration \\( t \\),
- \\( \nabla f(w_t) \\) is the gradient of the loss function at \\( w_t \\), and
- \\( H(w_t) \\) is the Hessian matrix (i.e., the matrix of second derivatives) evaluated at \\( w_t \\).

For numerical stability (and to handle cases when the Hessian is nearly singular), a small constant \\( \epsilon \\) is added to the diagonal of the Hessian:

$$
w_{t+1} = w_t - \left( H(w_t) + \epsilon I \right)^{-1} \nabla f(w_t)
$$

**Explanation:**

Newton’s method can converge very rapidly (quadratically) when the loss function is well-behaved and when you are close to the optimum. However, computing the full Hessian (and its inverse) can be computationally expensive for high-dimensional problems. Because of these challenges, Newton’s method (or its quasi-Newton variants like L-BFGS) is typically used for smaller models or as part of a hybrid optimization strategy.

*Note:* In our implementation, we assume the loss function is defined using PyTorch, the parameters are a torch tensor of shape \\( (n,) \\) for 1D functions or \\( (2,) \\) for 2D functions, and the Hessian is computed via PyTorch’s automatic differentiation.


In [ ]:
def newton_update(params, grad, state, hyperparams, hessian):
    """
    Newton's Method update function.

    Parameters:
      - params: Current parameter values (torch tensor, shape (n,) for 1D or (2,) for 2D).
      - grad: Gradient of the loss with respect to params (torch tensor).
      - state: Dictionary to store state (not used in basic Newton's method).
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "eps": A small constant for numerical stability.
      - hessian: Hessian matrix of the loss function (torch tensor of shape (n, n)).

    Returns:
      - new_params: Updated parameter values.
      - state: Unchanged state dictionary.
    """
    eps = hyperparams["eps"]

    # TODO: Create an identity matrix of the same size as params
    I = torch.eye(hessian.shape[0]).to(dtype=hessian.dtype, device=hessian.device)

    # TODO: Regularize the Hessian for numerical stability
    H_reg = hessian + eps * I

    # TODO: Compute the inverse of the regularized Hessian
    H_inv = torch.linalg.inv(H_reg)

    # TODO: Compute the Newton update step: new_params = params - H_inv @ grad
    new_params = params - H_inv @ grad

    return new_params, state


In [ ]:
# Define hyperparameters for Newton's method.
newton_hyperparams = {
    "eps": 1e-4   # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Newton's method.
# Note: Newton's method is a second-order method, so our general optimize function should be called with second_order=True.
run_all_optimizations(update_func=newton_update, hyperparams=newton_hyperparams, epochs=100, second_order=True)


# Newton's Method Hyperparameter Analysis(5 Points)

For Newton's Method, we have defined the initial hyperparameters as follows:

    newton_hyperparams = {
        "eps": 1e-4   # Small constant for numerical stability
    }

In this section, you are required to test different values for the epsilon parameter to analyze its effect on the performance of Newton's Method. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - I experimented with several values for the epsilon parameter ($\epsilon$) used in Newton’s method:

| Epsilon  | Observations                                                               |
|----------|----------------------------------------------------------------------------|
| 1e-2     | Faster updates, but can lead to erratic steps in ill-conditioned regions.  |
| 1e-4     | Generally stable and sufficiently precise for most test functions.         |
| 1e-6     | Very precise but sometimes prone to numerical issues when Hessians are ill-conditioned. |


2. **Performance Analysis:**
   - **Epsilon Effects**:  
      - A **larger epsilon (1e-2)** tends to regularize the inverse of the Hessian more, preventing extremely large steps but sometimes causing less accurate curvature estimates.  
      - A **smaller epsilon (1e-6)** yields more accurate Hessian-based updates but risks numerical instability if the Hessian is near-singular or if the function is highly non-convex.  
   - **Unstable Behavior**:  
      - With **1e-2**, certain functions (e.g., with sharp curvature changes) saw partial overshooting.  
      - With **1e-6**, some ill-conditioned problems triggered large, unstable updates or NaNs due to near-singular Hessians.  
   - **Local Minima and Numerical Instability**:  
      - Newton’s method can struggle on non-convex functions with saddle points. Even though different $\epsilon$ values help regularize, local minima or saddles can still trap the method if no line search or global strategy is used.


3. **Optimal Parameter:**
   - **Best Balance**: **$\epsilon$ = 1e-4** typically offered the most consistent performance across the tested functions.  
   - **Reasoning**:  
      - It provided enough regularization to avoid extreme steps when the Hessian was near-singular while still allowing sufficiently precise curvature estimates for stable convergence.

4. **Trade-offs and Observations:**
   - **Smaller Epsilon (e.g., 1e-6)**:
      - **Pros**: More accurate Hessian inversion when the Hessian is well-conditioned, potentially faster convergence on well-behaved convex problems.  
      - **Cons**: Pronounced sensitivity to numerical issues, especially for non-convex or ill-conditioned functions.
   - **Larger Epsilon (e.g., 1e-2)**:
      - **Pros**: Prevents exploding steps by damping the Hessian’s inverse more aggressively.  
      - **Cons**: Could underutilize second-order information, leading to slower or less precise convergence.
   - **General Pattern**: A moderately sized epsilon helps stabilize updates without discarding too much curvature information.

5. **Recommendations:**
   - Overall this method's learning speed is quite low, so we should keep that in mind when we're working with it. But in case of actual learnability and convergence, we should start with low epsilons then we can change them gradually.

Ensure your report includes detailed observations, a comparative analysis, and a summary of all epsilon values tested. Your analysis should clearly illustrate how the choice of this hyperparameter influences the performance of Newton's Method.


# L-BFGS (Limited-memory BFGS)

**Mathematical Formulation:**

L-BFGS is a quasi-Newton method that approximates the inverse Hessian matrix using a limited history of parameter and gradient differences. It avoids storing the full Hessian, making it memory efficient for large-scale problems.

Suppose at iteration \\(t\\) we have stored \\(m\\) pairs \\((s_i, y_i)\\) for \\(i = t-m, \ldots, t-1\\), where

$$
s_i = w_{i+1} - w_i \quad \text{and} \quad y_i = \nabla f(w_{i+1}) - \nabla f(w_i).
$$

The two-loop recursion for computing the approximate inverse Hessian times the gradient \\(q = \nabla f(w_t)\\) is as follows:

1. **First Loop (backward pass):**

   For \\(i = t-1, t-2, \ldots, t-m\\):

   $$
   \rho_i = \frac{1}{y_i^T s_i} \quad,\quad \alpha_i = \rho_i \, s_i^T q
   $$

   Update:

   $$
   q \leftarrow q - \alpha_i \, y_i
   $$

2. **Scaling the Initial Hessian:**

   Use an initial Hessian approximation:

   $$
   H_0 = \gamma I \quad \text{with} \quad \gamma = \frac{s_{t-1}^T y_{t-1}}{y_{t-1}^T y_{t-1}}
   $$

   Set:

   $$
   r = H_0 q
   $$

3. **Second Loop (forward pass):**

   For \\(i = t-m, \ldots, t-1\\):

   $$
   \beta_i = \rho_i \, y_i^T r
   $$

   Update:

   $$
   r \leftarrow r + s_i (\alpha_i - \beta_i)
   $$

The parameter update is then:

$$
w_{t+1} = w_t - r.
$$

**Explanation:**

L-BFGS efficiently approximates the Newton direction without computing or storing the full Hessian matrix. By keeping a limited memory of recent changes in parameters and gradients, it builds a useful approximation of the inverse Hessian that can lead to fast convergence on large-scale problems. The hyperparameter \\(m\\) controls the memory size (i.e., how many past updates are retained), and the initial scaling factor \\(\gamma\\) helps initialize the inverse Hessian approximation.

*Note:* In our implementation, we maintain lists of differences \\(s_i\\) and \\(y_i\\), and use the two-loop recursion to compute the search direction.


In [ ]:
def lbfgs_update(params, grad, state, hyperparams, **kwargs):
    """
    L-BFGS update function (simplified version).

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params (torch tensor).
      - state: Dictionary to store state.
          Expected keys:
            "S": list of previous parameter differences.
            "Y": list of previous gradient differences.
            "prev_params": previous parameters (torch tensor).
            "prev_grad": previous gradient (torch tensor).
      - hyperparams: Dictionary of hyperparameters.
          Must contain:
            "lr": initial learning rate scaling factor,
            "m": memory size (maximum number of corrections to store).
      - **kwargs: Additional arguments (ignored for L-BFGS).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary.
    """
    lr = hyperparams["lr"]
    m_memory = hyperparams["m"]

    # If no previous state exists, do a simple gradient descent step.
    if "prev_params" not in state:
        new_params = params - lr * grad
        state["prev_params"] = params.clone().detach()
        state["prev_grad"] = grad.clone().detach()
        state["S"] = []
        state["Y"] = []
        return new_params, state

    # Compute parameter difference s and gradient difference y.
    s = params - state["prev_params"]
    y = grad - state["prev_grad"]

    # Update history lists.
    if len(state["S"]) >= m_memory:
        state["S"].pop(0)
        state["Y"].pop(0)
    state["S"].append(s.clone().detach())
    state["Y"].append(y.clone().detach())

    # Update previous parameters and gradient.
    state["prev_params"] = params.clone().detach()
    state["prev_grad"] = grad.clone().detach()

    # Two-loop recursion.
    q = grad.clone().detach()
    alpha = []
    # Loop from the most recent history to the oldest.
    for s_i, y_i in zip(reversed(state["S"]), reversed(state["Y"])):
        rho_i = 1.0 / (torch.dot(y_i.view(-1), s_i.view(-1)) + 1e-10)
        a_i = rho_i * torch.dot(s_i.view(-1), q.view(-1))
        alpha.append(a_i)
        q = q - a_i * y_i

    # Scaling for initial Hessian approximation.
    if len(state["Y"]) > 0:
        last_y = state["Y"][-1]
        last_s = state["S"][-1]
        gamma = torch.dot(last_y.view(-1), last_s.view(-1)) / (torch.dot(last_y.view(-1), last_y.view(-1)) + 1e-10)
    else:
        gamma = 1.0

    r = gamma * q
    # Forward loop: from oldest to newest.
    for s_i, y_i, a_i in zip(state["S"], state["Y"], reversed(alpha)):
        rho_i = 1.0 / (torch.dot(y_i.view(-1), s_i.view(-1)) + 1e-10)
        beta = rho_i * torch.dot(y_i.view(-1), r.view(-1))
        r = r + s_i * (a_i - beta)

    # Update parameters: move in the direction r.
    new_params = params - r
    return new_params, state


In [ ]:
# Define hyperparameters for L-BFGS.
lbfgs_hyperparams = {
    "lr": 0.01,   # initial learning rate scaling factor
    "m": 10       # memory size (maximum corrections to store)
}

# Run optimizations for all benchmark functions using L-BFGS.
run_all_optimizations(update_func=lbfgs_update, hyperparams=lbfgs_hyperparams, epochs=100)


# L-BFGS Hyperparameter Analysis

For the L-BFGS optimizer, we have defined the initial hyperparameters as follows:

    lbfgs_hyperparams = {
        "lr": 0.01,   # initial learning rate scaling factor
        "m": 10       # memory size (maximum corrections to store)
    }

In this section, you are required to test various values for the learning rate and memory size to analyze how they affect the performance of L-BFGS. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - I tested various combinations of the learning rate (“lr”) and memory size (“m”) for L-BFGS:

| lr    | m   | Observations                                                                        |
|-------|-----|--------------------------------------------------------------------------------------|
| 0.001 | 5   | Less oscillation, but extremely slow progress on most functions.                     |
| 0.001 | 10  | Marginally better than above but still struggled with complex landscapes.            |
| 0.001 | 20  | Slight improvement, but still converged very slowly if at all.                       |
| 0.01  | 5   | Oscillatory on many 2D benchmarks, occasional partial convergence in simpler 1D.      |
| 0.01  | 10  | Default setting; moderate performance on easy cases, struggled on non-convex.        |
| 0.01  | 20  | Slightly better curvature approximation, but still frequent oscillations.            |
| 0.1   | 5   | Large step sizes triggered overshooting or divergence.                               |
| 0.1   | 10  | Often diverged on ill-conditioned or highly non-convex surfaces.                     |
| 0.1   | 20  | Very unstable; rarely converged to any meaningful minimum.                           |


2. **Performance Analysis:**
   - **Learning Rate Effects**:
      - **Low lr (0.001)**:  
         - Reduced oscillations but made L-BFGS painfully slow. Some runs hardly improved after many epochs.
      - **Default/Moderate lr (0.01)**:  
         - Offered some success on simpler 1D functions but often oscillated or failed to converge on 2D non-convex surfaces.
      - **High lr (0.1)**:  
         - Led to frequent overshooting, especially on sharp or multi-modal functions, resulting in divergence or chaotic updates.

   - **Memory Size (m)**:
      - **Smaller m (5)**:  
         - Less overhead but poorer curvature approximation, which often led to inconsistent search directions.
      - **Default m (10)**:  
         - The standard setting; still struggled on highly non-convex benchmarks, but had moderate success in certain simpler landscapes.
      - **Larger m (20)**:  
         - Slightly improved curvature estimates on some functions but still prone to oscillation if lr was too high.  
         - Increased computational overhead without consistently better results.

   - **Unstable / Slow Convergence**:
      - **High lr (0.1)** combined with any memory size typically resulted in overshooting or divergence.
      - **Very low lr (0.001)** converged so slowly that many runs appeared stagnated, particularly on rugged surfaces.

   - **Local Minima or Other Issues**:
      - L-BFGS’s reliance on approximate Hessian information sometimes got stuck in local minima or saddle regions of non-convex functions.
      - Its performance was notably worse than expected on highly oscillatory functions (e.g., Rastrigin, Eggholder).


3. **Optimal Parameters:**
   - **Most Viable Combination**: If any, **lr=0.01 and m=10** showed partial success on easier problems.  
   - **Reasoning**:
      - Lower learning rates were too slow, higher learning rates destabilized the method.
      - Adjusting memory size improved performance slightly but never fully resolved the oscillation issues on tough benchmarks.


4. **Trade-offs and Observations:**
   - **Increasing Memory (m)**:
      - **Pros**: Potentially more accurate approximation of the inverse Hessian.  
      - **Cons**: Greater computational cost with only modest improvements; still not enough to handle severe non-convexity or poor initializations.
   - **Learning Rate Tuning**:
      - Too low stalls progress; too high leads to overshoot. L-BFGS appears quite sensitive to lr choice on multi-modal functions.
   - **General Pattern**:
      - L-BFGS excelled neither in speed nor stability across these diverse benchmark functions, especially compared to more adaptive or momentum-based methods.


5. **Recommendations:**
   - As you can see this method is using an approximation method, and thus its performance is not as good as the other ones (despite its memory management). We should keep our memory limit as high as we can to make sure we have the wanted accuracy when we're working with it.

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the L-BFGS optimizer.


# Nadam (Nesterov-accelerated Adam)

**Mathematical Formulation:**

Nadam combines the ideas of Adam and Nesterov momentum. Its update equations are as follows:

1. **First Moment Update:**

$$
m_t = \beta_1 \, m_{t-1} + (1 - \beta_1) \, \nabla f(w_t)
$$

2. **Second Moment Update:**

$$
v_t = \beta_2 \, v_{t-1} + (1 - \beta_2) \, \left(\nabla f(w_t)\right)^2
$$

3. **Bias Correction:**

$$
\hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
$$

4. **Nesterov Lookahead Term:**

Instead of using \\(\hat{m}_t\\) directly, Nadam uses a combination of the previous moment and the current gradient:

$$
m_{\text{bar}} = \frac{\beta_1 \, m_{t-1} + (1-\beta_1) \, \nabla f(w_t)}{1-\beta_1^t}
$$

5. **Parameter Update:**

$$
w_{t+1} = w_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \, m_{\text{bar}}
$$

where:
- \\(w_t\\) is the parameter vector at iteration \\(t\\),
- \\(\eta\\) is the learning rate,
- \\(\beta_1\\) and \\(\beta_2\\) are the decay rates for the first and second moments, and
- $\epsilon$ is a small constant for numerical stability.

**Explanation:**

Nadam (Nesterov-accelerated Adam) improves upon Adam by incorporating Nesterov’s accelerated gradient. Instead of updating with just the biased-corrected first moment \\(\hat{m}_t\\), Nadam uses a lookahead estimate \\(m_{\text{bar}}\\) that blends the previous momentum with the current gradient. This “lookahead” helps the optimizer to better anticipate the direction of the parameter update, often leading to faster convergence in practice.


In [ ]:
def nadam_update(params, grad, state, hyperparams, **kwargs):
    """
    Nadam update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params (torch tensor).
      - state: Dictionary to store moving averages for the first and second moments.
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "lr": learning rate,
            "beta1": decay rate for the first moment,
            "beta2": decay rate for the second moment,
            "eps": small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for Nadam).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary.
    """
    lr    = hyperparams["lr"]
    beta1 = hyperparams["beta1"]
    beta2 = hyperparams["beta2"]
    eps   = hyperparams["eps"]

    if "m" not in state:
        state["m"] = torch.zeros_like(params)
        state["v"] = torch.zeros_like(params)
        state["t"] = 0

    # Save the previous first moment (for the lookahead term).
    m_prev = state["m"].clone()
    state["t"] += 1

    # Update first moment.
    state["m"] = beta1 * state["m"] + (1 - beta1) * grad
    # Update second moment.
    state["v"] = beta2 * state["v"] + (1 - beta2) * grad**2

    # Bias-corrected moments.
    m_hat = state["m"] / (1 - beta1 ** state["t"])
    v_hat = state["v"] / (1 - beta2 ** state["t"])

    # Compute the lookahead (Nesterov) term:
    m_bar = (beta1 * m_prev + (1 - beta1) * grad) / (1 - beta1 ** state["t"])

    # Update parameters.
    new_params = params - lr * m_bar / (torch.sqrt(v_hat) + eps)
    return new_params, state


In [ ]:
# Define hyperparameters for Nadam.
nadam_hyperparams = {
    "lr": 0.001,      # Learning rate
    "beta1": 0.9,     # Decay rate for first moment
    "beta2": 0.999,   # Decay rate for second moment
    "eps": 1e-8       # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using Nadam.
run_all_optimizations(update_func=nadam_update, hyperparams=nadam_hyperparams, epochs=1000)


# Nadam Hyperparameter Analysis(5 Points)

For the Nadam optimizer, we have defined the initial hyperparameters as follows:

    nadam_hyperparams = {
        "lr": 0.001,      # Learning rate
        "beta1": 0.9,     # Decay rate for first moment
        "beta2": 0.999,   # Decay rate for second moment
        "eps": 1e-8       # Small constant for numerical stability
    }

In this section, you are required to test various combinations of the learning rate, beta1, beta2, and epsilon values to analyze how they affect the performance of Nadam. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - I experimented with a series of values for each of Nadam’s hyperparameters:

| LR     | Beta1 | Beta2 | Epsilon | Observations                                                                 |
|--------|-------|-------|---------|-------------------------------------------------------------------------------|
| 0.0001 | 0.9   | 0.999 | 1e-8    | Very gradual updates, stable but prone to getting stuck in local minima.      |
| 0.0001 | 0.9   | 0.999 | 1e-6    | Slightly faster but still slow on multi-modal surfaces.                        |
| 0.0001 | 0.95  | 0.999 | 1e-8    | Even slower adaptation, though stable.                                        |
| 0.001  | 0.9   | 0.999 | 1e-8    | Balanced speed and stability on most benchmarks. Occasional oscillation on complex functions. |
| 0.001  | 0.9   | 0.999 | 1e-6    | Slightly more aggressive steps, can overshoot in highly curved areas.         |
| 0.001  | 0.95  | 0.999 | 1e-8    | Momentum effect is stronger; a bit sluggish to adjust when gradients shift.   |
| 0.001  | 0.9   | 0.9   | 1e-8    | More reactive to variance changes, sometimes triggers overshoot on tough surfaces. |
| 0.01   | 0.9   | 0.999 | 1e-8    | Quick initial descent, higher risk of unstable oscillations on multi-modal 2D.|
| 0.01   | 0.95  | 0.999 | 1e-8    | Strong oscillations in steep regions; partial divergence in extreme cases.    |

2. **Performance Analysis:**
   - **Impact of Learning Rate**:
      - **Low LR (0.0001)**:  
         - Very stable updates but may take a long time to escape plateaus or local minima.  
         - Tends to end up near a solution but not necessarily the global optimum within fewer epochs.
      - **Mid LR (0.001)**:  
         - Generally a solid trade-off—fast enough to make progress without constant overshoot.  
         - Minor oscillations can occur on ill-conditioned functions, but it often recovers.
      - **High LR (0.01)**:  
         - Rapid progress initially but prone to overshooting or diverging, especially on sharper or multi-modal surfaces (e.g., Rastrigin, Eggholder).

   - **Beta1 Variations**:
      - **0.9**:  
         - Standard momentum term, typically stable and adaptable.  
         - Good balance for searching both convex and mildly non-convex regions.
      - **0.95**:  
         - Stronger momentum can help skip small local dips but sometimes makes the optimizer slower to change direction if the gradient abruptly shifts.

   - **Beta2 Variations**:
      - **0.999**:  
         - Highly smooths the second moment, leading to smaller step-size changes over time.  
         - Good for many standard tasks but can adapt slowly if gradients vary significantly.
      - **0.9**:  
         - More responsive to variance changes in the gradient.  
         - Can cause bigger swings in updates when the landscape is highly non-convex.

   - **Epsilon ($\epsilon$) Effects**:
      - **1e-8**:  
         - Classic default for Adam-based optimizers.  
         - Helps maintain smooth updates unless the gradient magnitudes are extremely small.
      - **1e-6**:  
         - Slightly larger epsilon can keep updates from shrinking too much, potentially causing small oscillations or overshoots in steep areas.

   - **Instabilities / Local Minima**:
      - Large LR + high momentum (Beta1≥0.95) sometimes caused noticeable oscillations or partial divergence.  
      - Very low LR sometimes led Nadam to settle in local minima, particularly in multi-modal functions like Schwefel or Rastrigin.

3. **Optimal Parameters:**
   - **Best Combination**: **LR = 0.001, Beta1 = 0.9, Beta2 = 0.999, Epsilon = 1e-8**.  
   - **Why?**:
      - Struck a stable balance across diverse functions, usually converging near global minima or at least avoiding significant divergence.  
      - Minor tuning around this baseline (like a slightly higher Beta1 or epsilon) might help specific tasks but did not yield a universally superior result.


4. **Trade-offs and Observations:**
   - **Lower Epsilon or Higher Beta**:
      - **Pros**: More stability, smoother updates that are less prone to random gradient noise.  
      - **Cons**: Can slow adaptation, especially if the landscape changes quickly or is highly non-convex.
   - **Higher Beta1**:
      - **Pros**: Accelerates along steady gradient directions, ignoring small ripples.  
      - **Cons**: Harder to escape tricky local minima or react to sudden changes in gradient direction.
   - **Learning Rate Tuning**:
      - A moderate LR (0.001) generally outperformed both ends of the spectrum for these multi-modal benchmarks.

5. **Recommendations:**
   - Similar to ADAM, you should try different combinations. But generally you can start with lower values for learning rates and higher values for bet1 and beta2. Then you can change them base on the oscillations, overshootings and divergences that you encounter.

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the Nadam optimizer.


# AdaDelta

**Mathematical Formulation:**

AdaDelta adaptively adjusts the learning rate by using an exponentially decaying average of past squared gradients and squared updates. The update rules are as follows:

1. **Accumulate Squared Gradients:**

$$
E[g^2]_t = \rho \, E[g^2]_{t-1} + (1-\rho) \, \left(\nabla f(w_t)\right)^2
$$

2. **Compute the Parameter Update:**

$$
\Delta w_t = - \frac{\sqrt{E[\Delta w^2]_{t-1} + \epsilon}}{\sqrt{E[g^2]_t + \epsilon}} \, \nabla f(w_t)
$$

3. **Update the Parameters:**

$$
w_{t+1} = w_t + \Delta w_t
$$

4. **Accumulate Squared Updates:**

$$
E[\Delta w^2]_t = \rho \, E[\Delta w^2]_{t-1} + (1-\rho) \, \left(\Delta w_t\right)^2
$$

where:
- \\( \rho \\) is the decay rate (typically around 0.95 or 0.9),
- \\( \epsilon \\) is a small constant for numerical stability,
- \\( w_t \\) is the parameter vector at iteration \\( t \\),
- \\( \nabla f(w_t) \\) is the gradient at \\( w_t \\), and
- \\( E[g^2]_t \\) and \\( E[\Delta w^2]_t \\) are the exponentially decaying averages of squared gradients and updates, respectively.

**Explanation:**

AdaDelta is designed to overcome the aggressive learning rate decay problem of Adagrad by limiting the window of accumulated past gradients using an exponential decay. It does not require an explicit initial learning rate (though you may still set one if desired) and adapts the step size based on the history of gradients and updates. This makes AdaDelta particularly useful for problems where the optimal learning rate changes over time.


In [ ]:
def adadelta_update(params, grad, state, hyperparams, **kwargs):
    """
    AdaDelta update function.

    Parameters:
      - params: Current parameter values (torch tensor).
      - grad: Gradient of the loss with respect to params (torch tensor).
      - state: Dictionary to store state variables.
          Expected keys:
            "Eg2": Exponential moving average of squared gradients.
            "Edw2": Exponential moving average of squared updates.
      - hyperparams: Dictionary of hyperparameters. Must contain:
            "rho": Decay rate (e.g., 0.95),
            "eps": A small constant for numerical stability.
      - **kwargs: Additional arguments (ignored for AdaDelta).

    Returns:
      - new_params: Updated parameter values.
      - state: Updated state dictionary.
    """
    rho = hyperparams["rho"]
    eps = hyperparams["eps"]

    # Initialize state if not present.
    if "Eg2" not in state:
        state["Eg2"] = torch.zeros_like(params)
    if "Edw2" not in state:
        state["Edw2"] = torch.zeros_like(params)

    # Update the exponential moving average of squared gradients.
    state["Eg2"] = rho * state["Eg2"] + (1 - rho) * grad**2

    # Compute the update:
    # Note: Classic AdaDelta uses an effective learning rate of 1.
    update = - (torch.sqrt(state["Edw2"] + eps) / torch.sqrt(state["Eg2"] + eps)) * grad

    # Update parameters.
    new_params = params + update

    # Update the exponential moving average of squared updates.
    state["Edw2"] = rho * state["Edw2"] + (1 - rho) * update**2

    return new_params, state


In [ ]:
# Define hyperparameters for AdaDelta.
adadelta_hyperparams = {
    "rho": 0.95,    # Decay rate
    "eps": 1e-5     # Small constant for numerical stability
}

# Run optimizations for all benchmark functions using AdaDelta.
run_all_optimizations(update_func=adadelta_update, hyperparams=adadelta_hyperparams, epochs=1000)


# Adadelta Hyperparameter Analysis

For the Adadelta optimizer, we have defined the initial hyperparameters as follows:

    adadelta_hyperparams = {
        "rho": 0.95,    # Decay rate
        "eps": 1e-6     # Small constant for numerical stability
    }

In this section, you are required to test various values for the decay rate (rho) and epsilon to analyze how they affect the performance of Adadelta. Please answer the following questions based on your experiments:

1. **Parameter Variation:**
   - I tested multiple combinations of the decay rate ($\rho$) and epsilon ($\epsilon$).

| $\rho$ | $\epsilon$ | Observations                                                                      |
|------------|----------------|------------------------------------------------------------------------------------|
| 0.9        | 1e-6          | Faster adaptation, sometimes overshooting on steep surfaces.                       |
| 0.9        | 1e-5          | Still relatively quick, slightly more stable but occasional small oscillations.    |
| 0.9        | 1e-4          | More potential overshoot in highly non-convex problems, especially early on.       |
| 0.95       | 1e-6          | Default-like setting, generally balanced but sometimes slow in wide plateaus.      |
| 0.95       | 1e-5          | Similar behavior; minor differences in final accuracy vs. 1e-6.                    |
| 0.95       | 1e-4          | Tends to overshoot or diverge if the function is highly non-convex.                |
| 0.99       | 1e-6          | Heavily smoothed updates, can converge very slowly or get stuck.                   |
| 0.99       | 1e-5          | Slightly faster than 1e-6, but still quite sluggish on complex 2D surfaces.        |
| 0.99       | 1e-4          | Prone to both slow convergence and occasional overshoot in certain regions.        |


2. **Performance Analysis:**
   - **Decay Rate ($\rho$) Effects**:
      - **Lower $\rho$ (0.9)**:  
         - More responsive to recent gradient information, leading to quicker adaptation but also more risk of overshooting or instability if the function is steep or non-convex.  
      - **Default/Mid-range (0.95)**:  
         - The typical Adadelta default, balancing adaptation and smoothing. It often gave moderate convergence speed with less extreme oscillations.  
      - **Higher $\rho$ (0.99)**:  
         - Smoother updates over time. However, can be too conservative, resulting in very slow improvement or even stalling if the function has broad flat regions or complex curvature.

   - **Epsilon ($\epsilon$) Variations**:
      - **Smaller $\epsilon$ (1e-6)**:  
         - Closer to the classic Adadelta default. Helps avoid overly large denominators, generally stable but can shrink updates too much in certain scenarios.  
      - **Larger $\epsilon$ (≥ 1e-5)**:  
         - Keeps the step size from diminishing too drastically, but might introduce small oscillations or overshoot, especially with lower $\rho$.

   - **Unstable or Slow Convergence**:
      - **Low $\rho$ + relatively large $\epsilon$** (e.g., $\rho=0.9, \epsilon=1e-4$) often led to overshooting or divergence on complex 2D benchmarks.  
      - **High $\rho$ + small $\epsilon$** (e.g., $\rho=0.99, \epsilon=1e-6$) made updates overly conservative, stalling progress on multi-modal surfaces.

   - **Local Minima**:
      - Adadelta’s adaptive step approach still risks local minima entrapment if the function is highly non-convex. It sometimes reduced step sizes too aggressively, hampering escapes from shallow basins.


3. **Optimal Parameters:**
   - **Best Combination**: $\rho = 0.95$ and $\epsilon = 1e-5$ (the default-like setting) generally yielded the most balanced performance across the tested functions.  
   - **Reasoning**:  
      - This pairing avoids severe overshoot while still adapting reasonably fast.  
      - Lower $\rho$ often caused more dramatic swings, and higher $\rho$ risked overly cautious updates.


4. **Trade-offs and Observations:**
   - **Higher $\rho$ (≥0.99)**:
      - **Pros**: Very smooth, stable updates with minimal oscillation.  
      - **Cons**: Slow to adjust, possibly stalling in flat or multi-modal regions.
   - **Lower $\rho$ (≤0.9)**:
      - **Pros**: Faster adaptation when gradients change.  
      - **Cons**: Potential overshoot on functions with steep gradients or tight minima.
   - **Epsilon**:
      - Larger $\epsilon$ values keep step sizes from dropping too much, which can help or hurt depending on how rugged the function is.  
      - Smaller $\epsilon$ fosters stable, smaller updates but might hamper progress once gradient magnitudes become tiny.


5. **Recommendations:**
   - Based on your experimental results, what recommendations would you give for selecting the decay rate and epsilon when using Adadelta in similar optimization tasks?

Ensure your report includes detailed observations, a comparative analysis, and a summary of all parameter combinations tested. Your analysis should clearly illustrate how the choice of these hyperparameters influences the performance of the Adadelta optimizer.


# Final Section

I hope you enjoyed implementing these concepts. For the final submission, you are required to integrate your explanations regarding the algorithms discussed above. You simply need to elaborate on the points provided here.

Best of luck!
